In [ ]:
###########################################################################
#주식 매매 자동화 프로그램
#본 프로그램은 유안타증권(https://www.myasset.com) 사이트에서만 동작합니다
#본 프로그램에서 사용하는 매매규칙은 저자가 예를 들기 위해 만든 것으로
#사용자가 직접 변경하여 사용해야 하며
#본 프로그램을 사용하여 수익이나 손실이 발생할 경우
#모든 이익과 책임은 사용자게에 있음을 알려드립니다
#모두 부자 되세요~
#From 데이터쟁이 서진수 ^^
###########################################################################

#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import numpy as np
import pandas as pd   
import random
import os
import math
import cv2          # pip install opencv_python , pip install opencv-contrib-python
import pyautogui    # pip install pyautogui
import imutils      # pip install imutils

# Step 2.크롬 드라이버 지정 후 증권사 페이지에 접속하기
chrome_path = "c:/temp/chromedriver_85/chromedriver.exe"
url='https://www.myasset.com//'
driver = webdriver.Chrome(chrome_path)
driver.get(url)
driver.maximize_window()
time.sleep(10)

#Step 3.공인인증서 로그인하기   
#로그인 버튼 클릭
driver.find_element_by_xpath('//*[@id="gnb"]/div/div/div[1]/div[1]/div/span[1]/a[1]').click() 
time.sleep(10)

radio = driver.find_element_by_id("logoutTime05")   # 타임아웃시간을 6시간으로 설정
radio.click()
time.sleep(1)

driver.find_element_by_id('signLoginBtn').click()  # 공인인증서 로그인 버튼         

# 같은 그림 좌표 찾는 함수
def match_center_loc(screen, template):
    res = cv2.matchTemplate(screen, template, cv2.TM_CCOEFF_NORMED)
    min_val,max_val,min_loc, max_loc = cv2.minMaxLoc(res)
    top_left = max_loc
    h,w = template.shape[:2]
    x, y = int(top_left[0] + w/2), int(top_left[1] + h/2)
    return x,y

# 전체 화면 캡쳐하기
time.sleep(5)
image1 = pyautogui.screenshot()
image2 = cv2.cvtColor(np.array(image1), cv2.COLOR_RGB2BGR)
cv2.imwrite("c:\\data\\image\\myasset_full_screen_1.jpg", image2)
time.sleep(1)

screenshot = cv2.imread('c:\\data\\image\\myasset_full_screen_1.jpg', cv2.IMREAD_COLOR)

# 로그인 사용자명 선택용 이미지
template_1 = cv2.imread('c:\\data\\image\\myasset_name.jpg', cv2.IMREAD_COLOR)  

# 비밀번호 입력 창 선택용 이미지
template_2 = cv2.imread('c:\\data\\image\\myasset_password.jpg', cv2.IMREAD_COLOR)  

# 확인버튼 선택용 이미지
template_3 = cv2.imread('c:\\data\\image\\myasset_enter.jpg', cv2.IMREAD_COLOR) 

# 공인인증서 화면에서 사용자이름 선택하고 클릭
match_pos_1 = match_center_loc(screenshot, template_1)
pyautogui.moveTo(match_pos_1)
pyautogui.click()
time.sleep(1)

# 암호인증 창에 암호 입력하기
match_pos_2 = match_center_loc(screenshot, template_2)
pyautogui.moveTo(match_pos_2)
pyautogui.click()
password = '이곳에 여러분의 암호를 입력하세요'
pyautogui.typewrite(password)

# 암호 입력 후 확인 버튼 클릭하기
match_pos_3 = match_center_loc(screenshot, template_3)  
pyautogui.moveTo(match_pos_3)
pyautogui.click()
time.sleep(5)
  
# 팝업창 닫기
try:
    driver.find_element_by_id('LO_0000000_L1P1Btn').click()
except :
    print('')

time.sleep(1)

#Step 4. 주식 매매 창으로 들어가기
# 트레이딩 누르기
driver.find_element_by_xpath('//*[@id="ajaxGnb"]/ul/li[4]/a/span[1]').click()
time.sleep(1)

# 주식 누르기
driver.find_element_by_xpath('//*[@id="content"]/div/div/div[1]/div/ul/li[1]/a').click()
time.sleep(5)

# 전체 화면 캡쳐하기 - 화면 크기를 미리 조정하고 작업하기
image1 = pyautogui.screenshot()
image2 = cv2.cvtColor(np.array(image1), cv2.COLOR_RGB2BGR)
cv2.imwrite("c:\\data\\image\\myasset_full_screen_3.jpg", image2)
time.sleep(2)
screenshot = cv2.imread('c:\\data\\image\\myasset_full_screen_3.jpg', cv2.IMREAD_COLOR) 
jong_1 = cv2.imread('c:\\data\\image\\myasset_samsung_name.jpg', cv2.IMREAD_COLOR)     

#종목 클릭
match_jong_1 = match_center_loc(screenshot, jong_1)
pyautogui.moveTo(match_jong_1)
pyautogui.click()
time.sleep(1)
j_info = driver.find_element_by_xpath('//*[@id="W_0_F3000"]/div[2]/div/section[1]/div/ul[1]/li[2]/a')
driver.execute_script("arguments[0].click();", j_info)

#Step 5. 매수 / 매도하기
# 매수/ 매도 기본 값 설정하기
count = 1
masu = 1    #매수 여부 체크하는 변수

tot_buy_rate = float(0.35)  # 삼성전자의 3 단계 변경 합계, 매수할 때 사용할 조건 , 이 조건 이상되면 매수
tot_sell_rate = float(-0.35)  # 삼성전자의 3 단계 변경 합계, 매도할 때 사용할 조건 , 이 조건 이상이면 매도

buy_price='0'        # 처음 구입 가격 초기화 및 평균매수가격
buy_qty = 0          # 보유주식수
up_price = 0         # 수수료 계산해서 더해줄 가격
sell_price = str( int(buy_price) + 300 )  # 최소판매 예정 가격,구입가격 + 300 원
susu_rate = 1.0006   # 수수료비율

while 1:
    # 현재 시간 확인
    n = time.localtime()
    s = '%04d년 %02d월 %02d일 %02d시 %02d분 %02d초' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)
    s2 = '%02d:%02d:%02d' % (n.tm_hour, n.tm_min, n.tm_sec)

    #장 시작 체크
    if str(s2) <= '09:00:00' :
        print("현재시간은 %s 입니다" %s)
        print('아직 정규장이 시작되기 전입니다. 3분후 재시도 합니다 ^^')
        print("\n")
        time.sleep(1)
        # 화면 움직임 만들기 위해 매도가능 탭 클릭
        driver.find_element_by_xpath('//*[@id="_0_F3000interact-with-order"]/ul[1]/li[2]/a').click( )  
        time.sleep(1)
        
        # 화면 움직임 만들기 위해 매수가능 탭 클릭
        driver.find_element_by_xpath('//*[@id="_0_F3000interact-with-order"]/ul[1]/li[1]/a').click( )  
        time.sleep(180)
    elif str(s2) >= '15:19:00' : 
        print("현재시간은 %s 입니다" %s)
        print("정규장 종료 되었습니다. 5분후에 재시도합니다 ^^")
        print("\n")
        time.sleep(1)
        # 화면 움직임 만들기 위해 매도가능 탭 클릭
        driver.find_element_by_xpath('//*[@id="_0_F3000interact-with-order"]/ul[1]/li[2]/a').click( )  
        time.sleep(1)
        
        # 화면 움직임 만들기 위해 매수가능 탭 클릭
        driver.find_element_by_xpath('//*[@id="_0_F3000interact-with-order"]/ul[1]/li[1]/a').click( )  
        time.sleep(300)
    else :
        print("=" *45)
        print("%s 회차 조회 결과입니다=======================" %count )
        print("현재시간: %s" %s)
        print("=" *45)
       
        # 1회와 2회차는 가격 정보만 조회하고 3회차부터 가격 비교 후 매수/매도를 진행함
        if count == 1:            
            time.sleep(1)
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')

            j_name = soup.find('div','j-codeSearcher codeSearcher inline').find('span').get_text()
            print('1.종목이름:',j_name)

            price_rate_all = soup.find('div','panel').find('dl','dl-layout four').find_all('dd')           
            c_price_1 = price_rate_all[0].get_text().replace(",","")   #현재가격
            print('2.현재가격_1:',c_price_1)
            
            updown_price_1 = price_rate_all[1].get_text().replace(",","")   #변동 금액
            print('3.가격변동금액_1:',updown_price_1)

            c_rate_1 = price_rate_all[2].get_text().replace("%","") #등락율
            print('4.등락율_1:',c_rate_1)
            
            c_rate_2 = c_rate_1
           
            #가격 변동 비율 폭 계산
            c_rate_pok_1 = float(c_rate_2) - float(c_rate_1)
            print("5.1-3까지 최근 3회 변동폭 합계:%s" %round(c_rate_pok_1,2) )
            time.sleep(1)
            
            # 화면 움직임 만들기 위해 매도가능 탭 클릭
            driver.find_element_by_xpath('//*[@id="_0_F3000interact-with-order"]/ul[1]/li[2]/a').click( )  
            time.sleep(1)
            
            # 화면 움직임 만들기 위해 매수가능 탭 클릭
            driver.find_element_by_xpath('//*[@id="_0_F3000interact-with-order"]/ul[1]/li[1]/a').click( )  
            time.sleep(0.5)

        elif count ==2 :
            time.sleep(1)
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')

            j_name = soup.find('div','j-codeSearcher codeSearcher inline').find('span').get_text()
            print('1.종목이름:',j_name)

            price_rate_all = soup.find('div','panel').find('dl','dl-layout four').find_all('dd')

            c_price_2 = price_rate_all[0].get_text().replace(",","")   #현재가격
            print('2.현재가격_2:',c_price_2)

            updown_price_2 = price_rate_all[1].get_text().replace(",","")   #현재가격
            print('3.가격변동금액_2:',updown_price_2)

            c_rate_2 = price_rate_all[2].get_text().replace("%","") #등락율
            print('4.등락율_2:',c_rate_2)


            c_rate_pok_2 = float(c_rate_2) - float(c_rate_1)
            print("5.%s회 기준 최근 3회 변동폭 합계:%s" %(count,round(c_rate_pok_2,2) ) )

            time.sleep(1)
            # 화면 움직임 만들기 위해 매도가능 탭 클릭
            driver.find_element_by_xpath('//*[@id="_0_F3000interact-with-order"]/ul[1]/li[2]/a').click( )  
            time.sleep(1)
            
            # 화면 움직임 만들기 위해 매수가능 탭 클릭
            driver.find_element_by_xpath('//*[@id="_0_F3000interact-with-order"]/ul[1]/li[1]/a').click( )  
            time.sleep(0.5)

        elif count ==3 :
            time.sleep(1)
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')

            j_name = soup.find('div','j-codeSearcher codeSearcher inline').find('span').get_text()
            print('1.종목이름:',j_name)

            price_rate_all = soup.find('div','panel').find('dl','dl-layout four').find_all('dd')

            c_price_3 = price_rate_all[0].get_text().replace(",","")   #현재가격
            print('2.현재가격_3:',c_price_3)

            updown_price_3 = price_rate_all[1].get_text().replace(",","")   #현재가격
            print('3.가격변동금액_3:',updown_price_3)

            c_rate_3 = price_rate_all[2].get_text().replace("%","") #등락율
            print('4.등락율_3:',c_rate_3)

            c_rate_pok_3 = float(c_rate_3) - float(c_rate_1)
            
            #손절기준가격지정
            son_gijun_price = int(buy_price) *  0.05   #현재가의 -5% 면 손절
            son_price = int(buy_price) - int(c_price_3)
            
            print("5.%s회 기준 최근 3회 변동폭 합계:%s" %(count,round(c_rate_pok_3,2) ) )

            time.sleep(1)
            # 화면 움직임 만들기 위해 매도가능 탭 클릭
            driver.find_element_by_xpath('//*[@id="_0_F3000interact-with-order"]/ul[1]/li[2]/a').click( )  
            time.sleep(1)
            
            # 화면 움직임 만들기 위해 매수가능 탭 클릭
            driver.find_element_by_xpath('//*[@id="_0_F3000interact-with-order"]/ul[1]/li[1]/a').click( )  
            time.sleep(0.5)
            
            # 분할 매수를 위한 비율 지정
            price_gap = 0                
            if c_price_3 == buy_price or buy_price == '0' :
                price_gap = 0
            elif c_price_3 < buy_price and c_price_3 >= str(math.ceil(int(buy_price) * 0.99)) :  
                price_gap = 1  # 현재 가격이 구입 가격보다 1% 미만 하락했을때  1
            elif c_price_3 < buy_price and c_price_3 >= str(math.ceil(int(buy_price) * 0.97)) :  
                price_gap = 2  # 현재 가격이 구입 가격보다 1% - 3% 사이로 하락했을 때 2
            elif c_price_3 < buy_price and c_price_3 >= str(math.ceil(int(buy_price) * 0.95)) :  
                price_gap = 3  # 현재 가격이 구입가격보다 3% - 5% 사이로 하락했을 때  3
            elif c_price_3 < buy_price and c_price_3 <= str(math.ceil(int(buy_price) * 0.90))   : 
                price_gap = 4  # 현재 가격이 구입가격보다 10% 이하 하락했을 때  4

            elif c_price_3 > buy_price and c_price_3 <= str(math.ceil(int(buy_price) * 1.01)) :  
                price_gap = 5  # 현재 가격이 구입 가격보다 1% 미만 상승했을때
            elif c_price_3 > buy_price and c_price_3 <= str(math.ceil(int(buy_price) * 1.03)) :  
                price_gap = 6  # 현재 가격이 구입 가격보다 1% - 3% 사이로 상승했을 때
            elif c_price_3 > buy_price and c_price_3 <= str(math.ceil(int(buy_price) * 1.05)) :  
                price_gap = 7  # 현재 가격이 구입가격보다 3% - 5% 사이로 상승했을 때
            elif c_price_3 > buy_price and c_price_3 >= str(math.ceil(int(buy_price) * 1.10))  : 
                price_gap = 8  # 현재 가격이 구입가격보다 10% 사이로 상승했을 때

            # 매수 / 매도 주문하기
            # 매수 case 1 - 하락하다가 상승되는 시점 찾아서 매수
            if float(c_rate_1) > float(c_rate_2) and float(c_rate_2) < float(c_rate_3) and c_rate_pok_3 >= tot_buy_rate  :
                time.sleep(1)
                
                mesu_tab= driver.find_element_by_link_text('매수')  #매수 탭 클릭
                driver.execute_script("arguments[0].click();", mesu_tab)
                time.sleep(0.3)
                pwd = driver.find_element_by_name('passwd')
                pwd.clear()
                pwd.send_keys('이곳에 여러분의 HTS 암호를 입력하세요')
                jo_btn= driver.find_element_by_xpath('//*[@id="_0_F3000mesu"]/fieldset/div[2]/input[2]')  #조회 버튼
                driver.execute_script("arguments[0].click();", jo_btn)
                time.sleep(0.5)
                radio2 = driver.find_element_by_id("_0_F3000mesu_meme_gubun_radio2")   # 시장가 클릭
                radio2.click()

                time.sleep(0.5)
                radio3 = driver.find_element_by_id('_0_F3000mesu_meme_gubun_radio1')  # 지정가(보통) 클릭
                radio3.click( )

                time.sleep(0.5)

                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                time.sleep(0.5)
                
                #최대 매수가능 수량 구하기
                qty_1 = soup.find('table','deal_limit_price j-subInfo_tbl hide-on-mobile').select('tbody > tr > td')      
                try :
                    ava_qty = int(qty_1[0].get_text())
                except :
                    ava_qty = 0
                print('최대구입가능수량:' ,ava_qty)

                if ava_qty >= 1 and price_gap == 0 and ava_qty >= 5:
                    ava_count = math.floor(ava_qty * 0.3)   # 현재 수량에서 30% 구입   
                elif price_gap == 1 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.1)   # 남아있는 수량에서 10% 구입
                elif price_gap == 2 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.2)   # 남아있는 수량에서 20% 구입
                elif price_gap == 3 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.3)   # 남아있는 수량에서 30% 구입
                elif price_gap == 4 or ava_qty <= 5 :
                    ava_count = ava_qty                     # 남아있는 수량에서 전량 구입
                elif price_gap == 5 and ava_qty >= 5:
                    ava_count = math.floor(ava_qty * 0.1)   # 남아있는 수량에서 10% 구입
                elif price_gap == 6 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.2)   # 남아있는 수량에서 20% 구입
                elif price_gap == 7 or price_gap == 8 or ava_qty <= 5   :
                    ava_count = ava_qty                     # 남아있는 수량에서 전량 구입
                    
                if ava_count >= 1  :  
                    order_count = driver.find_element_by_name('order_cnt')
                    order_count.clear( )
                    time.sleep(0.5)
                    order_count.send_keys(ava_count)
                    masu_btn= driver.find_element_by_xpath('//*[@id="_0_F3000mesu"]/fieldset/div[8]/input')
                    driver.execute_script("arguments[0].click();", masu_btn)
                    time.sleep(0.5)
                    popup_masu_btn= driver.find_element_by_xpath('//*[@id="orderConfirm__0_F3000mesu"]/div[3]/div/button[1]/span')
                    driver.execute_script("arguments[0].click();", popup_masu_btn)
                    time.sleep(2)
                    driver.find_element_by_css_selector('span.ui-button-text').click()

                    close1 = pyautogui.screenshot()
                    close2 = cv2.cvtColor(np.array(close1), cv2.COLOR_RGB2BGR)
                    cv2.imwrite("c:\\data\\image\\close_full_screen_1.jpg", close2)
                    time.sleep(1)

                    close_3 = cv2.imread('c:\\data\\image\\close.jpg', cv2.IMREAD_COLOR)  #확인버튼 

                    match_close = match_center_loc(close2, close_3)
                    pyautogui.moveTo(match_close)
                    pyautogui.click()

                    print('%s 원으로 %s 수 매수 신청 완료했습니다' %(c_price_3,ava_count))
                    masu = 1
                                        
                    # 평균 매수가 구하기
                    #공식 : (보유주식수 * 보유주식매수가격) + (신규매수수량 * 신규매수가격) / (보유주식수 + 신규매수주식수)
                    c_stocks = buy_qty * int(buy_price)
                    n_stocks = ava_count * int(c_price_3)

                    buy_price = str( (math.floor((c_stocks + n_stocks) / (buy_qty + ava_count) * susu_rate) + up_price ))
                    print('평균 매수가격:' , buy_price)
                    
                    buy_qty = buy_qty + ava_count
                    
                    buy_price_rate = c_rate_3
                    sell_price = str( int(buy_price) + 50 )
                                                        
                    print('매수가 완료되어 5분 후 다시 시작합니다')
                    time.sleep(300)

            # 상승 상태 매수        
            elif float(c_rate_1) <= float(c_rate_2) and float(c_rate_2) < float(c_rate_3) and c_rate_pok_3 >= tot_buy_rate :
                time.sleep(1)
                
                #매수 탭 클릭
                mesu_tab= driver.find_element_by_link_text('매수')  
                driver.execute_script("arguments[0].click();", mesu_tab)
                time.sleep(0.3)
                pwd = driver.find_element_by_name('passwd')
                pwd.clear()
                pwd.send_keys('이곳에 여러분의 HTS 암호를 입력하세요')
                jo_btn= driver.find_element_by_xpath('//*[@id="_0_F3000mesu"]/fieldset/div[2]/input[2]')
                driver.execute_script("arguments[0].click();", jo_btn)
                time.sleep(0.5)
                
                # 시장가 클릭
                radio2 = driver.find_element_by_id("_0_F3000mesu_meme_gubun_radio2")   
                radio2.click()
                time.sleep(0.5)
                
                # 지정가(보통) 클릭
                radio3 = driver.find_element_by_id('_0_F3000mesu_meme_gubun_radio1')  
                radio3.click( )
                time.sleep(0.5)

                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                time.sleep(0.5)
                
                # 구입가능 최대수량 체크하기
                qty_1 = soup.find('table','deal_limit_price j-subInfo_tbl hide-on-mobile').select('tbody > tr > td')      
                try :
                    ava_qty = int(qty_1[0].get_text())
                except :
                    ava_qty = 0
                print('최대구입가능수량:' ,ava_qty)
                
                if ava_qty >= 1 and price_gap == 0 and ava_qty >= 5:
                    ava_count = math.floor(ava_qty * 0.3)  # 초기값으로 총 구입 가능 수량 중에서 30% 구입   
                elif price_gap == 1 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.1)  # 남아있는 수량에서 10% 구입
                elif price_gap == 2 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.2)  # 남아있는 수량에서 20% 구입
                elif price_gap == 3 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.3)  # 남아있는 수량에서 30% 구입
                elif price_gap == 4 or ava_qty <= 5 :
                    ava_count = ava_qty                    # 남아있는 수량에서 전량 구입
                elif price_gap == 5 and ava_qty >= 5:
                    ava_count = math.floor(ava_qty * 0.1)  # 남아있는 수량에서 10% 구입
                elif price_gap == 6 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.2)  # 남아있는 수량에서 20% 구입
                elif price_gap == 7 or price_gap == 8 or ava_qty <= 5   :
                    ava_count = ava_qty                    # 남아있는 수량에서 전량 구입
                    
                if ava_count >= 1 :
                    order_count = driver.find_element_by_name('order_cnt')
                    order_count.clear( )
                    time.sleep(0.5)
                    order_count.send_keys(ava_count)
                    
                    masu_btn= driver.find_element_by_xpath('//*[@id="_0_F3000mesu"]/fieldset/div[8]/input')
                    driver.execute_script("arguments[0].click();", masu_btn)
                    time.sleep(0.5)
                    
                    popup_masu_btn= driver.find_element_by_xpath('//*[@id="orderConfirm__0_F3000mesu"]/div[3]/div/button[1]/span')
                    driver.execute_script("arguments[0].click();", popup_masu_btn)
                    time.sleep(2)
                    driver.find_element_by_css_selector('span.ui-button-text').click()

                    close1 = pyautogui.screenshot()
                    close2 = cv2.cvtColor(np.array(close1), cv2.COLOR_RGB2BGR)
                    cv2.imwrite("c:\\data\\image\\close_full_screen_1.jpg", close2)
                    time.sleep(1)

                    close_3 = cv2.imread('c:\\data\\image\\close.jpg', cv2.IMREAD_COLOR)  #확인버튼 

                    match_close = match_center_loc(close2, close_3)
                    pyautogui.moveTo(match_close)
                    pyautogui.click()

                    print('%s 원으로 %s 주 매수 신청 완료했습니다' %(c_price_3,ava_count))
                    masu = 1
                    
                    # 평균 매수가 구하기 - 수수료 금액은 제외한 공식
                    #공식 : (보유주식수 * 보유주식매수가격) + (신규매수수량 * 신규매수가격) / (보유주식수 + 신규매수주식수)
                    c_stocks = buy_qty * int(buy_price)
                    n_stocks = ava_count * int(c_price_3)

                    buy_price = str( (math.floor((c_stocks + n_stocks) / (buy_qty + ava_count) * susu_rate) + up_price ))
                    print('평균 매수가격:' , buy_price)
                    
                    buy_qty = buy_qty + ava_count
                        
                    buy_price_rate = c_rate_3
                    sell_price = str( int(buy_price) + 50 )
                                        
                    print('매수가 완료되어 5분 후 다시 시작합니다')
                    time.sleep(300)

            # 매도하기
            elif float(c_rate_3) < float(c_rate_2) and c_rate_pok_3 <= tot_sell_rate and masu == 1 and sell_price < c_price_3:               
                madobtn = driver.find_element_by_xpath('//*[@id="_0_F3000interact-with-order"]/ul[1]/li[2]/a') 
                driver.execute_script("arguments[0].click();", madobtn)
                time.sleep(1)
                
                #매도 가능 수량 찾기
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')   
                try :
                    madojan = int( soup.find('tbody','CI-GRID-BODY-TABLE-TBODY').find('td',{'name': 'od_unit'}).get_text().replace(',','') )
                except :
                    madojan = 0
                
                if madojan >= 1 :
                    print("매도합니다~~~")
                    time.sleep(0.5)

                    madotab = driver.find_element_by_xpath('//*[@id="W_0_F3000"]/div[2]/div/section[4]/div/ul[1]/li[2]/a') 
                    driver.execute_script("arguments[0].click();", madotab)
                    time.sleep(0.5)
                    pwd = driver.find_element_by_name('passwd')
                    pwd.clear()
                    time.sleep(0.5)
                    pwd.send_keys('이곳에 여러분의 HTS 암호를 입력하세요')
                    time.sleep(0.5)
                    driver.find_element_by_xpath('//*[@id="_0_F3000medo"]/fieldset/div[2]/input[2]').click()  # 조회 클릭
                    time.sleep(0.5)
                    radio2 = driver.find_element_by_id("_0_F3000medo_meme_gubun_radio2")   # 시장가 클릭
                    radio2.click()

                    time.sleep(0.5)
                    radio3 = driver.find_element_by_id('_0_F3000medo_meme_gubun_radio1')  # 보통 클릭
                    radio3.click( )
                    time.sleep(1)

                    #매도 수량 입력
                    order_count = driver.find_element_by_name('order_cnt')
                    order_count.clear( )
                    time.sleep(0.5)
                    order_count.send_keys(madojan)
                    time.sleep(0.5)

                    madobtn2 = driver.find_element_by_xpath('//*[@id="_0_F3000medo"]/fieldset/div[9]/input')
                    driver.execute_script("arguments[0].click();", madobtn2) #매도 버튼 클릭
                    time.sleep(0.5)
                    
                    #팝업창 확인 클릭
                    madobtn3 = driver.find_element_by_xpath('//*[@id="orderConfirm__0_F3000medo"]/div[3]/div/button[1]/span') 
                    driver.execute_script("arguments[0].click();", madobtn3)

                    masu = 0
                    time.sleep(1)

                    close1 = pyautogui.screenshot()
                    close2 = cv2.cvtColor(np.array(close1), cv2.COLOR_RGB2BGR)
                    cv2.imwrite("c:\\data\\image\\close_full_screen_1.jpg", close2)
                    time.sleep(1)

                    close_3 = cv2.imread('c:\\data\\image\\close.jpg', cv2.IMREAD_COLOR)  #확인버튼 

                    match_close = match_center_loc(close2, close_3)
                    pyautogui.moveTo(match_close)
                    pyautogui.click()
                    time.sleep(1)

                    print('%s 원으로 %s주 매도 신청 완료했습니다' %(c_price_3,madojan) )
                    print('5분 뒤 다시 시작됩니다~잠시만 쉬세요~')
                    time.sleep(300)   # 매도 후 5 분 뒤 다시 매수 코드 작동
                    
                    buy_qty = 0
                    buy_price = '0'
                    buy_price_rate = 0.0
                    
            #손절하기
            elif son_gijun_price < son_price :
                madobtn = driver.find_element_by_xpath('//*[@id="_0_F3000interact-with-order"]/ul[1]/li[2]/a') #매도가능 클릭
                driver.execute_script("arguments[0].click();", madobtn)
                time.sleep(1)
                
                #매도 가능 수량 찾기
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')   
                try :
                    madojan = int( soup.find('tbody','CI-GRID-BODY-TABLE-TBODY').find('td',{'name': 'od_unit'}).get_text().replace(',','') )
                except :
                    madojan = 0
                
                if madojan >= 1 :
                    print("손절 매도합니다ㅠㅠㅠ")
                    time.sleep(0.5)
                    madotab = driver.find_element_by_xpath('//*[@id="W_0_F3000"]/div[2]/div/section[4]/div/ul[1]/li[2]/a') #매도 탭클릭
                    driver.execute_script("arguments[0].click();", madotab)
                    time.sleep(0.5)
                    pwd = driver.find_element_by_name('passwd')
                    pwd.clear()
                    time.sleep(0.5)
                    pwd.send_keys('이곳에 여러분의 HTS 암호를 입력하세요')
                    time.sleep(0.5)
                    driver.find_element_by_xpath('//*[@id="_0_F3000medo"]/fieldset/div[2]/input[2]').click()  # 조회 클릭
                    time.sleep(0.5)
                    radio2 = driver.find_element_by_id("_0_F3000medo_meme_gubun_radio2")   # 시장가 클릭
                    radio2.click()

                    time.sleep(0.5)
                    radio3 = driver.find_element_by_id('_0_F3000medo_meme_gubun_radio1')  # 보통 클릭
                    radio3.click( )
                    time.sleep(1)

                    #매도 수량 입력
                    order_count = driver.find_element_by_name('order_cnt')
                    order_count.clear( )
                    time.sleep(0.5)
                    order_count.send_keys(madojan)
                    #order_count.send_keys(1)
                    time.sleep(0.5)

                    madobtn2 = driver.find_element_by_xpath('//*[@id="_0_F3000medo"]/fieldset/div[9]/input')
                    driver.execute_script("arguments[0].click();", madobtn2) #매도 버튼 클릭
                    time.sleep(0.5)
                    
                    #팝업창 확인 클릭
                    madobtn3 = driver.find_element_by_xpath('//*[@id="orderConfirm__0_F3000medo"]/div[3]/div/button[1]/span') 
                    driver.execute_script("arguments[0].click();", madobtn3)

                    masu = 0
                    time.sleep(1)

                    close1 = pyautogui.screenshot()
                    close2 = cv2.cvtColor(np.array(close1), cv2.COLOR_RGB2BGR)
                    cv2.imwrite("c:\\data\\image\\close_full_screen_1.jpg", close2)
                    time.sleep(1)

                    close_3 = cv2.imread('c:\\data\\image\\close.jpg', cv2.IMREAD_COLOR)  #확인버튼 

                    match_close = match_center_loc(close2, close_3)
                    pyautogui.moveTo(match_close)
                    pyautogui.click()
                    time.sleep(1)

                    print('%s 원으로 %s주 손절 매도 신청 완료했습니다' %(c_price_3,madojan) ) 
                    print('5분 뒤 다시 시작됩니다~잠시만 쉬세요~')
                    time.sleep(300)   # 매도 후 5 분 뒤 다시 매수 코드 작동
                    buy_qty = 0
                    buy_price = '0'
                    buy_price_rate = 0.0

        elif count ==4 :
            time.sleep(1)
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')

            j_name = soup.find('div','j-codeSearcher codeSearcher inline').find('span').get_text()
            print('1.종목이름:',j_name)

            price_rate_all = soup.find('div','panel').find('dl','dl-layout four').find_all('dd')
            
            c_price_4 = price_rate_all[0].get_text().replace(",","")   #현재가격
            print('2.현재가격_4:',c_price_4)
            
            updown_price_4 = price_rate_all[1].get_text().replace(",","")   #현재가격
            print('3.가격변동금액_4:',updown_price_4)

            c_rate_4 = price_rate_all[2].get_text().replace("%","") #등락율
            print('4.등락율_4:',c_rate_4)


            c_rate_pok_4 = float(c_rate_4) - float(c_rate_2)
            
            #손절기준가격지정
            son_gijun_price = int(buy_price) *  0.05   #현재가의 -5% 면 손절
            son_price = int(buy_price) - int(c_price_4)


            print("5.%s회 기준 최근 3회 변동폭 차이:%s" %(count,round(c_rate_pok_4,2) ) )

            time.sleep(0.5)
            # 화면 움직임 만들기 위해 매도가능 탭 클릭
            driver.find_element_by_xpath('//*[@id="_0_F3000interact-with-order"]/ul[1]/li[2]/a').click( )  
            time.sleep(0.5)
            
            # 화면 움직임 만들기 위해 매수가능 탭 클릭
            driver.find_element_by_xpath('//*[@id="_0_F3000interact-with-order"]/ul[1]/li[1]/a').click( )  
            time.sleep(0.5)
            
            price_gap = 0
            
            # 하락했을 때 조건 설정
            if c_price_4 == buy_price or buy_price == '0' :
                price_gap = 0
            elif c_price_4 < buy_price and c_price_4 >= str(math.ceil(int(buy_price) * 0.99)) :  
                price_gap = 1  # 현재 가격이 구입 가격보다 1% 미만 하락했을때 1
            elif c_price_4 < buy_price and c_price_4 >= str(math.ceil(int(buy_price) * 0.97)) :  
                price_gap = 2  # 현재 가격이 구입 가격보다 1% - 3% 사이로 하락했을 때  2
            elif c_price_4 < buy_price and c_price_4 >= str(math.ceil(int(buy_price) * 0.95)) :  
                price_gap = 3  # 현재 가격이 구입가격보다 3% - 5% 사이로 하락했을 때  3
            elif c_price_4 < buy_price and c_price_4 <= str(math.ceil(int(buy_price) * 0.90))   : 
                price_gap = 4  # 현재 가격이 구입가격보다 10% 이하 하락했을 때  4
            
            # 상승했을 때 조건 설정
            elif c_price_4 > buy_price and c_price_4 <= str(math.ceil(int(buy_price) * 1.01)) :  
                price_gap = 5  # 현재 가격이 구입 가격보다 1% 미만 상승했을때 5
            elif c_price_4 > buy_price and c_price_4 <= str(math.ceil(int(buy_price) * 1.03)) :  
                price_gap = 6  # 현재 가격이 구입 가격보다 1% - 3% 사이로 상승했을 때  6
            elif c_price_4 > buy_price and c_price_4 <= str(math.ceil(int(buy_price) * 1.05)) :  
                price_gap = 7  # 현재 가격이 구입가격보다 3% - 5% 사이로 상승했을 때  7
            elif c_price_4 > buy_price and c_price_4 >= str(math.ceil(int(buy_price) * 1.10))  : 
                price_gap = 8  # 현재 가격이 구입가격보다 10% 사이로 상승했을 때  8

            # 매수 / 매도 주문하기
            # 하락하다가 반등하는 시점 찾아서 매수
            if float(c_rate_2) >= float(c_rate_3) and float(c_rate_3) < float(c_rate_4)  and c_rate_pok_4 >= tot_buy_rate :
                time.sleep(1)
                
                mesu_tab= driver.find_element_by_link_text('매수')  #매수 탭 클릭
                driver.execute_script("arguments[0].click();", mesu_tab)
                time.sleep(0.3)
                pwd = driver.find_element_by_name('passwd')
                pwd.clear()
                pwd.send_keys('이곳에 여러분의 HTS 암호를 입력하세요')
                jo_btn= driver.find_element_by_xpath('//*[@id="_0_F3000mesu"]/fieldset/div[2]/input[2]')
                driver.execute_script("arguments[0].click();", jo_btn)
                time.sleep(0.5)
                radio2 = driver.find_element_by_id("_0_F3000mesu_meme_gubun_radio2")   # 시장가 클릭
                radio2.click()

                time.sleep(0.5)
                radio3 = driver.find_element_by_id('_0_F3000mesu_meme_gubun_radio1')  # 지정가(보통) 클릭
                radio3.click( )

                time.sleep(0.5)

                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                time.sleep(0.5)

                qty_1 = soup.find('table','deal_limit_price j-subInfo_tbl hide-on-mobile').select('tbody > tr > td')   #최대수량    
                try :
                    ava_qty = int(qty_1[0].get_text())
                except :
                    ava_qty = 0
                print('최대구입가능수량:' ,ava_qty)
                
                if ava_qty >= 1 and price_gap == 0 and ava_qty >= 5:
                    ava_count = math.floor(ava_qty * 0.3)     
                elif price_gap == 1 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.1) 
                elif price_gap == 2 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.2)
                elif price_gap == 3 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.3)
                elif price_gap == 4 or ava_qty <= 5 :
                    ava_count = ava_qty
                elif price_gap == 5 and ava_qty >= 5:
                    ava_count = math.floor(ava_qty * 0.1)  
                elif price_gap == 6 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.2)
                elif price_gap == 7 or price_gap == 8 or ava_qty <= 5   :
                    ava_count = ava_qty
                    
                if ava_count >= 1 :

                    order_count = driver.find_element_by_name('order_cnt')
                    order_count.clear( )
                    time.sleep(0.5)
                    order_count.send_keys(ava_count)
                    masu_btn= driver.find_element_by_xpath('//*[@id="_0_F3000mesu"]/fieldset/div[8]/input')
                    driver.execute_script("arguments[0].click();", masu_btn)
                    time.sleep(0.5)

                    popup_masu_btn= driver.find_element_by_xpath('//*[@id="orderConfirm__0_F3000mesu"]/div[3]/div/button[1]/span')
                    driver.execute_script("arguments[0].click();", popup_masu_btn)
                    time.sleep(2)
                    driver.find_element_by_css_selector('span.ui-button-text').click()

                    close1 = pyautogui.screenshot()
                    close2 = cv2.cvtColor(np.array(close1), cv2.COLOR_RGB2BGR)
                    cv2.imwrite("c:\\data\\image\\close_full_screen_1.jpg", close2)
                    time.sleep(1)

                    close_3 = cv2.imread('c:\\data\\image\\close.jpg', cv2.IMREAD_COLOR)  #확인버튼 

                    match_close = match_center_loc(close2, close_3)
                    pyautogui.moveTo(match_close)
                    pyautogui.click()

                    print('%s 원으로 %s 수 매수 신청 완료했습니다' %(c_price_4,ava_count))
                    masu = 1
                    
                    # 평균 매수가 구하기
                    #공식 : (보유주식수 * 보유주식매수가격) + (신규매수수량 * 신규매수가격) / (보유주식수 + 신규매수주식수)
                    c_stocks = buy_qty * int(buy_price)
                    n_stocks = ava_count * int(c_price_4)

                    buy_price = str( (math.floor((c_stocks + n_stocks) / (buy_qty + ava_count) * susu_rate) + up_price ))
                    print('평균 매수가격:' , buy_price)
                    
                    buy_qty = buy_qty + ava_count
                    
                    buy_price_rate = c_rate_4
                    sell_price = str( int(buy_price) + 50 )
                                                            
                    print('매수가 완료되어 5분 후 다시 시작합니다')
                    time.sleep(300)
                    
            # 상승 상태 매수        
            elif float(c_rate_2) <= float(c_rate_3) and float(c_rate_3) < float(c_rate_4) and c_rate_pok_4 >= tot_buy_rate :
                time.sleep(1)
                mesu_tab= driver.find_element_by_link_text('매수')  #매수 탭 클릭
                driver.execute_script("arguments[0].click();", mesu_tab)
                time.sleep(1)
                pwd = driver.find_element_by_name('passwd')
                pwd.clear()
                pwd.send_keys('이곳에 여러분의 HTS 암호를 입력하세요')
                jo_btn= driver.find_element_by_xpath('//*[@id="_0_F3000mesu"]/fieldset/div[2]/input[2]')
                driver.execute_script("arguments[0].click();", jo_btn)
                time.sleep(0.5)
                radio2 = driver.find_element_by_id("_0_F3000mesu_meme_gubun_radio2")   # 시장가 클릭
                radio2.click()

                time.sleep(0.5)
                radio3 = driver.find_element_by_id('_0_F3000mesu_meme_gubun_radio1')  # 지정가(보통) 클릭
                radio3.click( )
                time.sleep(0.5)

                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                time.sleep(0.5)

                qty_1 = soup.find('table','deal_limit_price j-subInfo_tbl hide-on-mobile').select('tbody > tr > td')   #최대수량    
                try :
                    ava_qty = int(qty_1[0].get_text())
                except :
                    ava_qty = 0
                print('최대구입가능수량:' ,ava_qty)
                    
                if ava_qty >= 1 and price_gap == 0 and ava_qty >= 5:
                    ava_count = math.floor(ava_qty * 0.3)      
                elif price_gap == 1 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.1)  
                elif price_gap == 2 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.2)
                elif price_gap == 3 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.3)
                elif price_gap == 4 or ava_qty <= 5 :
                    ava_count = ava_qty
                elif price_gap == 5 and ava_qty >= 5:
                    ava_count = math.floor(ava_qty * 0.1)  
                elif price_gap == 6 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.2)
                elif price_gap == 7 or price_gap == 8 or ava_qty <= 5   :
                    ava_count = ava_qty

                if ava_count >= 1 :

                    order_count = driver.find_element_by_name('order_cnt')
                    order_count.clear( )
                    time.sleep(0.5)
                    order_count.send_keys(ava_count)
                    masu_btn= driver.find_element_by_xpath('//*[@id="_0_F3000mesu"]/fieldset/div[8]/input')
                    driver.execute_script("arguments[0].click();", masu_btn)
                    time.sleep(0.5)
                    popup_masu_btn= driver.find_element_by_xpath('//*[@id="orderConfirm__0_F3000mesu"]/div[3]/div/button[1]/span')
                    driver.execute_script("arguments[0].click();", popup_masu_btn)
                    time.sleep(2)
                    driver.find_element_by_css_selector('span.ui-button-text').click()

                    close1 = pyautogui.screenshot()
                    close2 = cv2.cvtColor(np.array(close1), cv2.COLOR_RGB2BGR)
                    cv2.imwrite("c:\\data\\image\\close_full_screen_1.jpg", close2)
                    time.sleep(1)

                    close_3 = cv2.imread('c:\\data\\image\\close.jpg', cv2.IMREAD_COLOR)  #확인버튼 

                    match_close = match_center_loc(close2, close_3)
                    pyautogui.moveTo(match_close)
                    pyautogui.click()

                    print('%s 원으로 %s 수 매수 신청 완료했습니다' %(c_price_4,ava_count))
                    masu = 1
                    
                    # 평균 매수가 구하기
                    #공식 : (보유주식수 * 보유주식매수가격) + (신규매수수량 * 신규매수가격) / (보유주식수 + 신규매수주식수)
                    c_stocks = buy_qty * int(buy_price)
                    n_stocks = ava_count * int(c_price_4)

                    buy_price = str( (math.floor((c_stocks + n_stocks) / (buy_qty + ava_count) * susu_rate) + up_price ))
                    print('평균 매수가격:' , buy_price)
                    
                    buy_qty = buy_qty + ava_count
                    
                    buy_price_rate = c_rate_4
                    sell_price = str( int(buy_price) + 50 )
                                                            
                    print('매수가 완료되어 5분 후 다시 시작합니다')
                    time.sleep(300)

            # 매도하기
            elif float(c_rate_4) < float(c_rate_3) and c_rate_pok_4 <= tot_sell_rate and masu == 1 and sell_price < c_price_4 :           
                madobtn = driver.find_element_by_xpath('//*[@id="_0_F3000interact-with-order"]/ul[1]/li[2]/a') #매도가능 클릭
                driver.execute_script("arguments[0].click();", madobtn)
                time.sleep(1)
                
                #매도 가능 수량 찾기
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')            
                try :
                    madojan = int( soup.find('tbody','CI-GRID-BODY-TABLE-TBODY').find('td',{'name': 'od_unit'}).get_text().replace(',','') )
                except :
                    madojan = 0
                
                if madojan >= 1 :
                    print("매도합니다~~~")
                    time.sleep(0.5)
                    madotab = driver.find_element_by_xpath('//*[@id="W_0_F3000"]/div[2]/div/section[4]/div/ul[1]/li[2]/a') #매도 탭클릭
                    driver.execute_script("arguments[0].click();", madotab)
                    time.sleep(0.5)
                    pwd = driver.find_element_by_name('passwd')
                    pwd.clear()
                    time.sleep(0.5)
                    pwd.send_keys('이곳에 여러분의 HTS 암호를 입력하세요')
                    time.sleep(0.5)
                    driver.find_element_by_xpath('//*[@id="_0_F3000medo"]/fieldset/div[2]/input[2]').click()  # 조회 클릭
                    time.sleep(0.5)
                    radio2 = driver.find_element_by_id("_0_F3000medo_meme_gubun_radio2")   # 시장가 클릭
                    radio2.click()

                    time.sleep(0.5)
                    radio3 = driver.find_element_by_id('_0_F3000medo_meme_gubun_radio1')  # 보통 클릭
                    radio3.click( )
                    time.sleep(1)

                    #매도 수량 입력
                    order_count = driver.find_element_by_name('order_cnt')
                    order_count.clear( )
                    time.sleep(0.5)
                    order_count.send_keys(madojan)
                    time.sleep(0.5)

                    madobtn2 = driver.find_element_by_xpath('//*[@id="_0_F3000medo"]/fieldset/div[9]/input')
                    driver.execute_script("arguments[0].click();", madobtn2) #매도 버튼 클릭
                    time.sleep(0.5)
                    
                    #매도 팝업창 확인 클릭
                    madobtn3 = driver.find_element_by_xpath('//*[@id="orderConfirm__0_F3000medo"]/div[3]/div/button[1]/span') 
                    driver.execute_script("arguments[0].click();", madobtn3)

                    masu = 0
                    time.sleep(1)

                    close1 = pyautogui.screenshot()
                    close2 = cv2.cvtColor(np.array(close1), cv2.COLOR_RGB2BGR)
                    cv2.imwrite("c:\\data\\image\\close_full_screen_1.jpg", close2)
                    time.sleep(1)

                    close_3 = cv2.imread('c:\\data\\image\\close.jpg', cv2.IMREAD_COLOR)  #확인버튼 

                    match_close = match_center_loc(close2, close_3)
                    pyautogui.moveTo(match_close)
                    pyautogui.click()
                    time.sleep(1)

                    print('%s 원으로 %s주 매도 신청 완료했습니다' %(c_price_4,madojan) ) 
                    print('5분 뒤 다시 시작됩니다~잠시만 쉬세요~')
                    time.sleep(300)   # 매도 후 5 분 뒤 다시 매수 코드 작동
                    buy_qty = 0
                    buy_price = '0'
                    buy_price_rate = 0.0
                    
            #손절하기
            elif son_gijun_price < son_price :
                madobtn = driver.find_element_by_xpath('//*[@id="_0_F3000interact-with-order"]/ul[1]/li[2]/a') #매도가능 클릭
                driver.execute_script("arguments[0].click();", madobtn)
                time.sleep(1)
                
                #매도 가능 수량 찾기
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')   
                try :
                    madojan = int( soup.find('tbody','CI-GRID-BODY-TABLE-TBODY').find('td',{'name': 'od_unit'}).get_text().replace(',','') )
                except :
                    madojan = 0
                
                if madojan >= 1 :
                    print("손절 매도합니다ㅠㅠㅠ")
                    time.sleep(0.5)
                    madotab = driver.find_element_by_xpath('//*[@id="W_0_F3000"]/div[2]/div/section[4]/div/ul[1]/li[2]/a') #매도 탭클릭
                    driver.execute_script("arguments[0].click();", madotab)
                    time.sleep(0.5)
                    pwd = driver.find_element_by_name('passwd')
                    pwd.clear()
                    time.sleep(0.5)
                    pwd.send_keys('이곳에 여러분의 HTS 암호를 입력하세요')
                    time.sleep(0.5)
                    driver.find_element_by_xpath('//*[@id="_0_F3000medo"]/fieldset/div[2]/input[2]').click()  # 조회 클릭
                    time.sleep(0.5)
                    radio2 = driver.find_element_by_id("_0_F3000medo_meme_gubun_radio2")   # 시장가 클릭
                    radio2.click()

                    time.sleep(0.5)
                    radio3 = driver.find_element_by_id('_0_F3000medo_meme_gubun_radio1')  # 보통 클릭
                    radio3.click( )
                    time.sleep(1)

                    #매도 수량 입력
                    order_count = driver.find_element_by_name('order_cnt')
                    order_count.clear( )
                    time.sleep(0.5)
                    order_count.send_keys(madojan)
                    time.sleep(0.5)

                    madobtn2 = driver.find_element_by_xpath('//*[@id="_0_F3000medo"]/fieldset/div[9]/input')
                    driver.execute_script("arguments[0].click();", madobtn2) #매도 버튼 클릭
                    time.sleep(0.5)
                    
                    #매도 팝업창 확인 클릭
                    madobtn3 = driver.find_element_by_xpath('//*[@id="orderConfirm__0_F3000medo"]/div[3]/div/button[1]/span') 
                    driver.execute_script("arguments[0].click();", madobtn3)

                    masu = 0
                    time.sleep(1)

                    close1 = pyautogui.screenshot()
                    close2 = cv2.cvtColor(np.array(close1), cv2.COLOR_RGB2BGR)
                    cv2.imwrite("c:\\data\\image\\close_full_screen_1.jpg", close2)
                    time.sleep(1)

                    close_3 = cv2.imread('c:\\data\\image\\close.jpg', cv2.IMREAD_COLOR)  #확인버튼 

                    match_close = match_center_loc(close2, close_3)
                    pyautogui.moveTo(match_close)
                    pyautogui.click()
                    time.sleep(1)

                    print('%s 원으로 %s주 손절 매도 신청 완료했습니다' %(c_price_4,madojan) ) 
                    print('5분 뒤 다시 시작됩니다~잠시만 쉬세요~')
                    time.sleep(300)   # 매도 후 5 분 뒤 다시 매수 코드 작동
                    buy_qty = 0
                    buy_price = '0'
                    buy_price_rate = 0.0

        elif count == 5 : 
            time.sleep(1)
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')

            j_name = soup.find('div','j-codeSearcher codeSearcher inline').find('span').get_text()
            print('1.종목이름:',j_name)

            price_rate_all = soup.find('div','panel').find('dl','dl-layout four').find_all('dd')
            
            c_price_5 = price_rate_all[0].get_text().replace(",","")   
            print('2.현재가격_5:',c_price_5)
            
            updown_price_5 = price_rate_all[1].get_text().replace(",","")  
            print('3.가격변동금액_5:',updown_price_5)

            c_rate_5 = price_rate_all[2].get_text().replace("%","") 
            print('4.등락율_5:',c_rate_5)


            c_rate_pok_5 = float(c_rate_5) - float(c_rate_3)

            #손절기준가격지정
            son_gijun_price = int(buy_price) *  0.05   #현재가의 -5% 면 손절
            son_price = int(buy_price) - int(c_price_5)


            print("5.%s회 기준 최근 3회 변동폭 차이:%s" %(count,round(c_rate_pok_5,2) ) )

            time.sleep(1)
            # 화면 움직임 만들기 위해 매도가능 탭 클릭
            driver.find_element_by_xpath('//*[@id="_0_F3000interact-with-order"]/ul[1]/li[2]/a').click( )  
            time.sleep(1)
            
            # 화면 움직임 만들기 위해 매수가능 탭 클릭
            driver.find_element_by_xpath('//*[@id="_0_F3000interact-with-order"]/ul[1]/li[1]/a').click( )  
            time.sleep(0.5)
            
            price_gap = 0
                
            if c_price_5 == buy_price or buy_price == '0' :
                price_gap = 0
            elif c_price_5< buy_price and c_price_5 >= str(math.ceil(int(buy_price) * 0.99)) :  
                price_gap = 1  # 현재 가격이 구입 가격보다 1% 미만 하락했을때
            elif c_price_5 < buy_price and c_price_5 >= str(math.ceil(int(buy_price) * 0.97)) :  
                price_gap = 2  # 현재 가격이 구입 가격보다 1% - 3% 사이로 하락했을 때
            elif c_price_5 < buy_price and c_price_5 >= str(math.ceil(int(buy_price) * 0.95)) :  
                price_gap = 3  # 현재 가격이 구입가격보다 3% - 5% 사이로 하락했을 때
            elif c_price_5 < buy_price and c_price_5 <= str(math.ceil(int(buy_price) * 0.90))   : 
                price_gap = 4  # 현재 가격이 구입가격보다 10% 이하 하락했을 때

            elif c_price_5 > buy_price and c_price_5 <= str(math.ceil(int(buy_price) * 1.01)) :  
                price_gap = 5  # 현재 가격이 구입 가격보다 1% 미만 상승했을때
            elif c_price_5 > buy_price and c_price_5 <= str(math.ceil(int(buy_price) * 1.03)) :  
                price_gap = 6  # 현재 가격이 구입 가격보다 1% - 3% 사이로 상승했을 때
            elif c_price_5 > buy_price and c_price_5 <= str(math.ceil(int(buy_price) * 1.05)) :  
                price_gap = 7  # 현재 가격이 구입가격보다 3% - 5% 사이로 상승했을 때
            elif c_price_5 > buy_price and c_price_5 >= str(math.ceil(int(buy_price) * 1.10))  : 
                price_gap = 8  # 현재 가격이 구입가격보다 10% 사이로 상승했을 때

            # 매수 / 매도 주문하기
            # 하락 하다가 상승 전환시 매수하기
            if float(c_rate_3) >= float(c_rate_4) and float(c_rate_4) < float(c_rate_5) and c_rate_pok_5 >= tot_buy_rate :
                time.sleep(1)                
                mesu_tab= driver.find_element_by_link_text('매수')  #매수 탭 클릭
                driver.execute_script("arguments[0].click();", mesu_tab)
                time.sleep(0.3)
                pwd = driver.find_element_by_name('passwd')
                pwd.clear()
                pwd.send_keys('이곳에 여러분의 HTS 암호를 입력하세요')
                jo_btn= driver.find_element_by_xpath('//*[@id="_0_F3000mesu"]/fieldset/div[2]/input[2]')
                driver.execute_script("arguments[0].click();", jo_btn)
                time.sleep(0.5)
                radio2 = driver.find_element_by_id("_0_F3000mesu_meme_gubun_radio2")   # 시장가 클릭
                radio2.click()

                time.sleep(0.5)
                radio3 = driver.find_element_by_id('_0_F3000mesu_meme_gubun_radio1')  # 지정가(보통) 클릭
                radio3.click( )
                time.sleep(0.5)

                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                time.sleep(0.5)

                qty_1 = soup.find('table','deal_limit_price j-subInfo_tbl hide-on-mobile').select('tbody > tr > td')   #최대수량    
                try :
                    ava_qty = int(qty_1[0].get_text())
                except :
                    ava_qty = 0
                print('최대구입가능수량:' ,ava_qty)
                
                if ava_qty >= 1 and price_gap == 0 and ava_qty >= 5:
                    ava_count = math.floor(ava_qty * 0.3)     
                elif price_gap == 1 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.1)  
                elif price_gap == 2 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.2)
                elif price_gap == 3 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.3)
                elif price_gap == 4 or ava_qty <= 5 :
                    ava_count = ava_qty
                elif price_gap == 5 and ava_qty >= 5:
                    ava_count = math.floor(ava_qty * 0.1)  
                elif price_gap == 6 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.2)
                elif price_gap == 7 or price_gap == 8 or ava_qty <= 5   :
                    ava_count = ava_qty

                if ava_count >= 1 :
                    order_count = driver.find_element_by_name('order_cnt')
                    order_count.clear( )
                    time.sleep(0.5)
                    order_count.send_keys(ava_count)

                    masu_btn= driver.find_element_by_xpath('//*[@id="_0_F3000mesu"]/fieldset/div[8]/input')
                    driver.execute_script("arguments[0].click();", masu_btn)
                    time.sleep(0.5)
                    popup_masu_btn= driver.find_element_by_xpath('//*[@id="orderConfirm__0_F3000mesu"]/div[3]/div/button[1]/span')
                    driver.execute_script("arguments[0].click();", popup_masu_btn)
                    time.sleep(2)
                    driver.find_element_by_css_selector('span.ui-button-text').click()

                    close1 = pyautogui.screenshot()
                    close2 = cv2.cvtColor(np.array(close1), cv2.COLOR_RGB2BGR)
                    cv2.imwrite("c:\\data\\image\\close_full_screen_1.jpg", close2)
                    time.sleep(1)

                    close_3 = cv2.imread('c:\\data\\image\\close.jpg', cv2.IMREAD_COLOR)  #확인버튼 

                    match_close = match_center_loc(close2, close_3)
                    pyautogui.moveTo(match_close)
                    pyautogui.click()

                    print('%s 원으로 %s 수 매수 신청 완료했습니다' %(c_price_5,ava_count))
                    masu = 1
                    
                    # 평균 매수가 구하기
                    #공식 : (보유주식수 * 보유주식매수가격) + (신규매수수량 * 신규매수가격) / (보유주식수 + 신규매수주식수)
                    c_stocks = buy_qty * int(buy_price)
                    n_stocks = ava_count * int(c_price_5)

                    buy_price = str( (math.floor((c_stocks + n_stocks) / (buy_qty + ava_count) * susu_rate) + up_price ))
                    print('평균 매수가격:' , buy_price)
                    
                    buy_qty = buy_qty + ava_count
                    
                    buy_price_rate = c_rate_5
                    sell_price = str( int(buy_price) + 50 )
                                        
                    print('매수가 완료되어 5분 후 다시 시작합니다')
                    time.sleep(300)

            # 상승 상태 매수        
            elif float(c_rate_3) <= float(c_rate_4) and float(c_rate_4) < float(c_rate_5) and c_rate_pok_5 >= tot_buy_rate :
                time.sleep(1)
                mesu_tab= driver.find_element_by_link_text('매수')  #매수 탭 클릭
                driver.execute_script("arguments[0].click();", mesu_tab)
                time.sleep(0.3)

                pwd = driver.find_element_by_name('passwd')
                pwd.clear()
                pwd.send_keys('이곳에 여러분의 HTS 암호를 입력하세요')
                jo_btn= driver.find_element_by_xpath('//*[@id="_0_F3000mesu"]/fieldset/div[2]/input[2]')
                driver.execute_script("arguments[0].click();", jo_btn)
                time.sleep(0.5)
                radio2 = driver.find_element_by_id("_0_F3000mesu_meme_gubun_radio2")   # 시장가 클릭
                radio2.click()

                time.sleep(0.5)
                radio3 = driver.find_element_by_id('_0_F3000mesu_meme_gubun_radio1')  # 지정가(보통) 클릭
                radio3.click( )
                time.sleep(0.5)

                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                time.sleep(0.5)

                qty_1 = soup.find('table','deal_limit_price j-subInfo_tbl hide-on-mobile').select('tbody > tr > td')   #최대수량    
                try :
                    ava_qty = int(qty_1[0].get_text())
                except :
                    ava_qty = 0
                print('최대구입가능수량:' ,ava_qty)
                    
                if ava_qty >= 1 and price_gap == 0 and ava_qty >= 5:
                    ava_count = math.floor(ava_qty * 0.3)    
                elif price_gap == 1 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.1)  
                elif price_gap == 2 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.2)
                elif price_gap == 3 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.3)
                elif price_gap == 4 or ava_qty <= 5 :
                    ava_count = ava_qty
                elif price_gap == 5 and ava_qty >= 5:
                    ava_count = math.floor(ava_qty * 0.1)  
                elif price_gap == 6 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.2)
                elif price_gap == 7 or price_gap == 8 or ava_qty <= 5   :
                    ava_count = ava_qty

                if ava_count >= 1 :
                    order_count = driver.find_element_by_name('order_cnt')
                    order_count.clear( )
                    time.sleep(0.5)
                    order_count.send_keys(ava_count)
                    #driver.find_element_by_xpath('//*[@id="_0_F3000mesu"]/fieldset/div[8]/input').click( )  #매수클릭
                    masu_btn= driver.find_element_by_xpath('//*[@id="_0_F3000mesu"]/fieldset/div[8]/input')
                    driver.execute_script("arguments[0].click();", masu_btn)
                    time.sleep(0.5)
                    #driver.find_element_by_xpath('//*[@id="orderConfirm__0_F3000mesu"]/div[3]/div/button[1]/span').click() #팝업창 매수 클릭
                    popup_masu_btn= driver.find_element_by_xpath('//*[@id="orderConfirm__0_F3000mesu"]/div[3]/div/button[1]/span')
                    driver.execute_script("arguments[0].click();", popup_masu_btn)
                    time.sleep(2)
                    driver.find_element_by_css_selector('span.ui-button-text').click()

                    close1 = pyautogui.screenshot()
                    close2 = cv2.cvtColor(np.array(close1), cv2.COLOR_RGB2BGR)
                    cv2.imwrite("c:\\data\\image\\close_full_screen_1.jpg", close2)
                    time.sleep(1)

                    close_3 = cv2.imread('c:\\data\\image\\close.jpg', cv2.IMREAD_COLOR)  #확인버튼 

                    match_close = match_center_loc(close2, close_3)
                    pyautogui.moveTo(match_close)
                    pyautogui.click()

                    print('%s 원으로 %s 수 매수 신청 완료했습니다' %(c_price_5,ava_count))
                    masu = 1
                    
                    # 평균 매수가 구하기
                    #공식 : (보유주식수 * 보유주식매수가격) + (신규매수수량 * 신규매수가격) / (보유주식수 + 신규매수주식수)
                    c_stocks = buy_qty * int(buy_price)
                    n_stocks = ava_count * int(c_price_5)

                    buy_price = str( (math.floor((c_stocks + n_stocks) / (buy_qty + ava_count) * susu_rate) + up_price ))
                    print('평균 매수가격:' , buy_price)
                    
                    buy_qty = buy_qty + ava_count
                    
                    buy_price_rate = c_rate_5
                    sell_price = str( int(buy_price) + 50 )
                    
                                        
                    print('매수가 완료되어 5분 후 다시 시작합니다')
                    time.sleep(300)

            # 매도하기
            elif float(c_rate_5) < float(c_rate_4) and c_rate_pok_5 <= tot_sell_rate and masu == 1 and sell_price < c_price_5:
            
                madobtn = driver.find_element_by_xpath('//*[@id="_0_F3000interact-with-order"]/ul[1]/li[2]/a') #매도가능 클릭
                driver.execute_script("arguments[0].click();", madobtn)
                time.sleep(1)
                
                #매도 가능 수량 찾기
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')            
                try :
                    madojan = int( soup.find('tbody','CI-GRID-BODY-TABLE-TBODY').find('td',{'name': 'od_unit'}).get_text().replace(',','') )
                except :
                    madojan = 0
                
                if madojan >= 1 :
                    print("매도합니다~~~")
                    time.sleep(0.5)
                    madotab = driver.find_element_by_xpath('//*[@id="W_0_F3000"]/div[2]/div/section[4]/div/ul[1]/li[2]/a') #매도 탭클릭
                    driver.execute_script("arguments[0].click();", madotab)
                    time.sleep(0.5)
                    pwd = driver.find_element_by_name('passwd')
                    pwd.clear()
                    time.sleep(0.5)
                    pwd.send_keys('이곳에 여러분의 HTS 암호를 입력하세요')
                    time.sleep(0.5)
                    driver.find_element_by_xpath('//*[@id="_0_F3000medo"]/fieldset/div[2]/input[2]').click()  # 조회 클릭
                    time.sleep(0.5)
                    radio2 = driver.find_element_by_id("_0_F3000medo_meme_gubun_radio2")   # 시장가 클릭
                    radio2.click()

                    time.sleep(0.5)
                    radio3 = driver.find_element_by_id('_0_F3000medo_meme_gubun_radio1')  # 보통 클릭
                    radio3.click( )
                    #driver.find_element_by_xpath('//*[@id="_0_F3000medo"]/fieldset/div[5]/div/input[3]').click() # 현재가보다 한단계 + 키 누르기
                    time.sleep(1)

                    #매도 수량 입력
                    order_count = driver.find_element_by_name('order_cnt')
                    order_count.clear( )
                    time.sleep(0.5)
                    order_count.send_keys(madojan)
                    #order_count.send_keys(1)
                    time.sleep(0.5)

                    madobtn2 = driver.find_element_by_xpath('//*[@id="_0_F3000medo"]/fieldset/div[9]/input')
                    driver.execute_script("arguments[0].click();", madobtn2) #매도 버튼 클릭
                    time.sleep(0.5)

                    madobtn3 = driver.find_element_by_xpath('//*[@id="orderConfirm__0_F3000medo"]/div[3]/div/button[1]/span') #팝업창 확인 클릭
                    driver.execute_script("arguments[0].click();", madobtn3)

                    masu = 0
                    time.sleep(1)

                    close1 = pyautogui.screenshot()
                    close2 = cv2.cvtColor(np.array(close1), cv2.COLOR_RGB2BGR)
                    cv2.imwrite("c:\\data\\image\\close_full_screen_1.jpg", close2)
                    time.sleep(1)

                    close_3 = cv2.imread('c:\\data\\image\\close.jpg', cv2.IMREAD_COLOR)  #확인버튼 

                    match_close = match_center_loc(close2, close_3)
                    pyautogui.moveTo(match_close)
                    pyautogui.click()
                    time.sleep(1)

                    print('%s 원으로 %s주 매도 신청 완료했습니다' %(c_price_5,madojan) ) 
                    print('5분 뒤 다시 시작됩니다~잠시만 쉬세요~')
                    time.sleep(300)   # 매도 후 5 분 뒤 다시 매수 코드 작동
                    
                    buy_qty = 0
                    buy_price = '0'
                    buy_price_rate = 0.0
                    
            #손절하기
            elif son_gijun_price < son_price :
                madobtn = driver.find_element_by_xpath('//*[@id="_0_F3000interact-with-order"]/ul[1]/li[2]/a') #매도가능 클릭
                driver.execute_script("arguments[0].click();", madobtn)
                time.sleep(1)
                
                #매도 가능 수량 찾기
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')   
                try :
                    madojan = int( soup.find('tbody','CI-GRID-BODY-TABLE-TBODY').find('td',{'name': 'od_unit'}).get_text().replace(',','') )
                except :
                    madojan = 0
                
                if madojan >= 1 :
                    print("손절 매도합니다ㅠㅠㅠ")
                    time.sleep(0.5)
                    madotab = driver.find_element_by_xpath('//*[@id="W_0_F3000"]/div[2]/div/section[4]/div/ul[1]/li[2]/a') #매도 탭클릭
                    driver.execute_script("arguments[0].click();", madotab)
                    time.sleep(0.5)
                    pwd = driver.find_element_by_name('passwd')
                    pwd.clear()
                    time.sleep(0.5)
                    pwd.send_keys('이곳에 여러분의 HTS 암호를 입력하세요')
                    time.sleep(0.5)
                    driver.find_element_by_xpath('//*[@id="_0_F3000medo"]/fieldset/div[2]/input[2]').click()  # 조회 클릭
                    time.sleep(0.5)
                    radio2 = driver.find_element_by_id("_0_F3000medo_meme_gubun_radio2")   # 시장가 클릭
                    radio2.click()

                    time.sleep(0.5)
                    radio3 = driver.find_element_by_id('_0_F3000medo_meme_gubun_radio1')  # 보통 클릭
                    radio3.click( )
                    #driver.find_element_by_xpath('//*[@id="_0_F3000medo"]/fieldset/div[5]/div/input[3]').click() # 현재가보다 한단계 + 키 누르기
                    time.sleep(1)

                    #매도 수량 입력
                    order_count = driver.find_element_by_name('order_cnt')
                    order_count.clear( )
                    time.sleep(0.5)
                    order_count.send_keys(madojan)
                    #order_count.send_keys(1)
                    time.sleep(0.5)

                    madobtn2 = driver.find_element_by_xpath('//*[@id="_0_F3000medo"]/fieldset/div[9]/input')
                    driver.execute_script("arguments[0].click();", madobtn2) #매도 버튼 클릭
                    time.sleep(0.5)

                    madobtn3 = driver.find_element_by_xpath('//*[@id="orderConfirm__0_F3000medo"]/div[3]/div/button[1]/span') #팝업창 확인 클릭
                    driver.execute_script("arguments[0].click();", madobtn3)

                    masu = 0
                    time.sleep(1)

                    close1 = pyautogui.screenshot()
                    close2 = cv2.cvtColor(np.array(close1), cv2.COLOR_RGB2BGR)
                    cv2.imwrite("c:\\data\\image\\close_full_screen_1.jpg", close2)
                    time.sleep(1)

                    close_3 = cv2.imread('c:\\data\\image\\close.jpg', cv2.IMREAD_COLOR)  #확인버튼 

                    match_close = match_center_loc(close2, close_3)
                    pyautogui.moveTo(match_close)
                    pyautogui.click()
                    time.sleep(1)

                    print('%s 원으로 %s주 손절 매도 신청 완료했습니다' %(c_price_5,madojan) ) 
                    print('5분 뒤 다시 시작됩니다~잠시만 쉬세요~')
                    time.sleep(300)   # 매도 후 5 분 뒤 다시 매수 코드 작동
                    buy_qty = 0
                    buy_price = '0'
                    buy_price_rate = 0.0

        elif count == 6 : 
            time.sleep(1)
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')

            j_name = soup.find('div','j-codeSearcher codeSearcher inline').find('span').get_text()
            print('1.종목이름:',j_name)

            price_rate_all = soup.find('div','panel').find('dl','dl-layout four').find_all('dd')
            
            c_price_6 = price_rate_all[0].get_text().replace(",","")   #현재가격
            print('2.현재가격_6:',c_price_6)
            
            updown_price_6 = price_rate_all[1].get_text().replace(",","")   #현재가격
            print('3.가격변동금액_6:',updown_price_6)

            c_rate_6 = price_rate_all[2].get_text().replace("%","") #등락율
            print('4.등락율_6:',c_rate_6)


            c_rate_pok_6 = float(c_rate_6) - float(c_rate_4)
            
            #손절기준가격지정
            son_gijun_price = int(buy_price) *  0.05   #현재가의 -5% 면 손절
            son_price = int(buy_price) - int(c_price_6)

            print("5.%s회 기준 최근 3회 변동폭 차이:%s" %(count,round(c_rate_pok_6,2) ) )

            time.sleep(0.5)
            driver.find_element_by_xpath('//*[@id="_0_F3000interact-with-order"]/ul[1]/li[2]/a').click( )  # 화면 움직임 만들기 위해 매도가능 탭 클릭
            time.sleep(0.5)
            driver.find_element_by_xpath('//*[@id="_0_F3000interact-with-order"]/ul[1]/li[1]/a').click( )  # 화면 움직임 만들기 위해 매수가능 탭 클릭
            time.sleep(0.5)
            
            price_gap = 0
                
            if c_price_6 == buy_price or buy_price == '0' :
                price_gap = 0
            elif c_price_6 < buy_price and c_price_6 >= str(math.ceil(int(buy_price) * 0.99)) :  # 현재 가격이 구입 가격보다 1% 미만 하락했을때
                price_gap = 1
            elif c_price_6 < buy_price and c_price_6 >= str(math.ceil(int(buy_price) * 0.97)) :  # 현재 가격이 구입 가격보다 1% - 3% 사이로 하락했을 때
                price_gap = 2
            elif c_price_6 < buy_price and c_price_6 >= str(math.ceil(int(buy_price) * 0.95)) :  # 현재 가격이 구입가격보다 3% - 5% 사이로 하락했을 때
                price_gap = 3
            elif c_price_6 < buy_price and c_price_6 <= str(math.ceil(int(buy_price) * 0.90))   : # 현재 가격이 구입가격보다 10% 이하 하락했을 때
                price_gap = 4

            elif c_price_6 > buy_price and c_price_6 <= str(math.ceil(int(buy_price) * 1.01)) :  # 현재 가격이 구입 가격보다 1% 미만 상승했을때
                price_gap = 5
            elif c_price_6 > buy_price and c_price_6 <= str(math.ceil(int(buy_price) * 1.03)) :  # 현재 가격이 구입 가격보다 1% - 3% 사이로 상승했을 때
                price_gap = 6
            elif c_price_6 > buy_price and c_price_6 <= str(math.ceil(int(buy_price) * 1.05)) :  # 현재 가격이 구입가격보다 3% - 5% 사이로 상승했을 때
                price_gap = 7
            elif c_price_6 > buy_price and c_price_6 >= str(math.ceil(int(buy_price) * 1.10))  : # 현재 가격이 구입가격보다 10% 사이로 상승했을 때
                price_gap = 8

            # 매수 / 매도 주문하기
            # 하락 하다가 상승 전환될때 매수하기
            if float(c_rate_4) >= float(c_rate_5) and float(c_rate_5) < float(c_rate_6) and c_rate_pok_6  >= tot_buy_rate :
                time.sleep(1)
                mesu_tab= driver.find_element_by_link_text('매수')  #매수 탭 클릭
                driver.execute_script("arguments[0].click();", mesu_tab)
                time.sleep(0.3)
                    
                driver.find_element_by_xpath('//*[@id="W_0_F3000"]/div[2]/div/section[4]/div/ul[1]/li[1]').click( ) #매수버튼 클릭
                pwd = driver.find_element_by_name('passwd')
                pwd.clear()
                pwd.send_keys('이곳에 여러분의 HTS 암호를 입력하세요')
                #driver.find_element_by_xpath('//*[@id="_0_F3000mesu"]/fieldset/div[2]/input[2]').click()  # 조회 클릭
                jo_btn= driver.find_element_by_xpath('//*[@id="_0_F3000mesu"]/fieldset/div[2]/input[2]')
                driver.execute_script("arguments[0].click();", jo_btn)
                time.sleep(0.5)
                radio2 = driver.find_element_by_id("_0_F3000mesu_meme_gubun_radio2")   # 시장가 클릭
                radio2.click()

                time.sleep(0.5)
                radio3 = driver.find_element_by_id('_0_F3000mesu_meme_gubun_radio1')  # 지정가(보통) 클릭
                radio3.click( )

                #time.sleep(0.5)
                #driver.find_element_by_xpath('//*[@id="_0_F3000mesu"]/fieldset/div[5]/div/input[3]').click( ) # 현재가보다 한단계 + 키 누르기
                #time.sleep(0.3)
                #driver.find_element_by_xpath('//*[@id="_0_F3000mesu"]/fieldset/div[5]/div/input[3]').click( ) # 현재가보다 한단계 + 키 누르기
                #driver.find_element_by_xpath('//*[@id="_0_F3000mesu"]/fieldset/div[5]/div/input[1]').click() # 현재가보다 한단계 - 키 누르기
                time.sleep(0.5)

                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                time.sleep(0.5)

                qty_1 = soup.find('table','deal_limit_price j-subInfo_tbl hide-on-mobile').select('tbody > tr > td')   #최대수량    
                try :
                    ava_qty = int(qty_1[0].get_text())
                except :
                    ava_qty = 0
                print('최대구입가능수량:' ,ava_qty)
                
                if ava_qty >= 1 and price_gap == 0 and ava_qty >= 5:
                    ava_count = math.floor(ava_qty * 0.3)   # 30% 구입   
                elif price_gap == 1 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.1)  # 남아있는 금액에서 10% 구입:
                elif price_gap == 2 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.2)
                elif price_gap == 3 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.3)
                elif price_gap == 4 or ava_qty <= 5 :
                    ava_count = ava_qty
                elif price_gap == 5 and ava_qty >= 5:
                    ava_count = math.floor(ava_qty * 0.1)  # 남아있는 금액에서 30% 구입
                elif price_gap == 6 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.2)
                elif price_gap == 7 or price_gap == 8 or ava_qty <= 5   :
                    ava_count = ava_qty

                if ava_count >= 1 :

                    order_count = driver.find_element_by_name('order_cnt')
                    order_count.clear( )
                    time.sleep(0.5)
                    order_count.send_keys(ava_count)
                    #driver.find_element_by_xpath('//*[@id="_0_F3000mesu"]/fieldset/div[8]/input').click( )  #매수클릭
                    masu_btn= driver.find_element_by_xpath('//*[@id="_0_F3000mesu"]/fieldset/div[8]/input')
                    driver.execute_script("arguments[0].click();", masu_btn)
                    time.sleep(0.5)
                    #driver.find_element_by_xpath('//*[@id="orderConfirm__0_F3000mesu"]/div[3]/div/button[1]/span').click() #팝업창 매수 클릭
                    popup_masu_btn= driver.find_element_by_xpath('//*[@id="orderConfirm__0_F3000mesu"]/div[3]/div/button[1]/span')
                    driver.execute_script("arguments[0].click();", popup_masu_btn)
                    time.sleep(2)
                    driver.find_element_by_css_selector('span.ui-button-text').click()

                    close1 = pyautogui.screenshot()
                    close2 = cv2.cvtColor(np.array(close1), cv2.COLOR_RGB2BGR)
                    cv2.imwrite("c:\\data\\image\\close_full_screen_1.jpg", close2)
                    time.sleep(1)

                    close_3 = cv2.imread('c:\\data\\image\\close.jpg', cv2.IMREAD_COLOR)  #확인버튼 

                    match_close = match_center_loc(close2, close_3)
                    pyautogui.moveTo(match_close)
                    pyautogui.click()

                    print('%s 원으로 %s 수 매수 신청 완료했습니다' %(c_price_6,ava_count))
                    masu = 1
                    
                    # 평균 매수가 구하기
                    #공식 : (보유주식수 * 보유주식매수가격) + (신규매수수량 * 신규매수가격) / (보유주식수 + 신규매수주식수)
                    c_stocks = buy_qty * int(buy_price)
                    n_stocks = ava_count * int(c_price_6)

                    buy_price = str( (math.floor((c_stocks + n_stocks) / (buy_qty + ava_count) * susu_rate) + up_price ))
                    print('평균 매수가격:' , buy_price)
                    
                    buy_qty = buy_qty + ava_count
                    
                    buy_price_rate = c_rate_6
                    sell_price = str( int(buy_price) + 50 )
                    
                                        
                    print('매수가 완료되어 5분 후 다시 시작합니다')
                    time.sleep(300)

            # 상승 상태 매수        
            elif float(c_rate_4) <= float(c_rate_5) and float(c_rate_5) < float(c_rate_6) and c_rate_pok_6 >= tot_buy_rate:
                time.sleep(1)
                #driver.find_element_by_xpath('//*[@id="W_0_F3000"]/div[2]/div/section[4]/div/ul[1]/li[1]').click( ) #매수버튼 클릭

                mesu_tab= driver.find_element_by_link_text('매수')  #매수 탭 클릭
                driver.execute_script("arguments[0].click();", mesu_tab)
                time.sleep(0.3)

                pwd = driver.find_element_by_name('passwd')
                pwd.clear()
                pwd.send_keys('이곳에 여러분의 HTS 암호를 입력하세요')
                #driver.find_element_by_xpath('//*[@id="_0_F3000mesu"]/fieldset/div[2]/input[2]').click()  # 조회 클릭
                jo_btn= driver.find_element_by_xpath('//*[@id="_0_F3000mesu"]/fieldset/div[2]/input[2]')
                driver.execute_script("arguments[0].click();", jo_btn)
                time.sleep(0.5)
                radio2 = driver.find_element_by_id("_0_F3000mesu_meme_gubun_radio2")   # 시장가 클릭
                radio2.click()

                time.sleep(0.5)
                radio3 = driver.find_element_by_id('_0_F3000mesu_meme_gubun_radio1')  # 지정가(보통) 클릭
                radio3.click( )

                #time.sleep(0.5)
                #driver.find_element_by_xpath('//*[@id="_0_F3000mesu"]/fieldset/div[5]/div/input[3]').click( ) # 현재가보다 한단계 + 키 누르기
                #time.sleep(0.3)
                #driver.find_element_by_xpath('//*[@id="_0_F3000mesu"]/fieldset/div[5]/div/input[3]').click( ) # 현재가보다 한단계 + 키 누르기
                #driver.find_element_by_xpath('//*[@id="_0_F3000mesu"]/fieldset/div[5]/div/input[1]').click() # 현재가보다 한단계 - 키 누르기
                time.sleep(0.5)

                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                time.sleep(0.5)

                qty_1 = soup.find('table','deal_limit_price j-subInfo_tbl hide-on-mobile').select('tbody > tr > td')   #최대수량    
                try :
                    ava_qty = int(qty_1[0].get_text())
                except :
                    ava_qty = 0
                print('최대구입가능수량:' ,ava_qty)
                    
                if ava_qty >= 1 and price_gap == 0 and ava_qty >= 5:
                    ava_count = math.floor(ava_qty * 0.3)   # 30% 구입   
                elif price_gap == 1 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.1)  # 남아있는 금액에서 10% 구입:
                elif price_gap == 2 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.2)
                elif price_gap == 3 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.3)
                elif price_gap == 4 or ava_qty <= 5 :
                    ava_count = ava_qty
                elif price_gap == 5 and ava_qty >= 5:
                    ava_count = math.floor(ava_qty * 0.1)  # 남아있는 금액에서 30% 구입
                elif price_gap == 6 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.2)
                elif price_gap == 7 or price_gap == 8 or ava_qty <= 5   :
                    ava_count = ava_qty

                if ava_count >= 1 :

                    order_count = driver.find_element_by_name('order_cnt')
                    order_count.clear( )
                    time.sleep(0.5)
                    order_count.send_keys(ava_count)
                    #driver.find_element_by_xpath('//*[@id="_0_F3000mesu"]/fieldset/div[8]/input').click( )  #매수클릭
                    masu_btn= driver.find_element_by_xpath('//*[@id="_0_F3000mesu"]/fieldset/div[8]/input')
                    driver.execute_script("arguments[0].click();", masu_btn)
                    time.sleep(0.5)
                    #driver.find_element_by_xpath('//*[@id="orderConfirm__0_F3000mesu"]/div[3]/div/button[1]/span').click() #팝업창 매수 클릭
                    popup_masu_btn= driver.find_element_by_xpath('//*[@id="orderConfirm__0_F3000mesu"]/div[3]/div/button[1]/span')
                    driver.execute_script("arguments[0].click();", popup_masu_btn)
                    time.sleep(2)
                    driver.find_element_by_css_selector('span.ui-button-text').click()

                    close1 = pyautogui.screenshot()
                    close2 = cv2.cvtColor(np.array(close1), cv2.COLOR_RGB2BGR)
                    cv2.imwrite("c:\\data\\image\\close_full_screen_1.jpg", close2)
                    time.sleep(1)

                    close_3 = cv2.imread('c:\\data\\image\\close.jpg', cv2.IMREAD_COLOR)  #확인버튼 

                    match_close = match_center_loc(close2, close_3)
                    pyautogui.moveTo(match_close)
                    pyautogui.click()

                    print('%s 원으로 %s 수 매수 신청 완료했습니다' %(c_price_6,ava_count))
                    masu = 1
                    
                    # 평균 매수가 구하기
                    #공식 : (보유주식수 * 보유주식매수가격) + (신규매수수량 * 신규매수가격) / (보유주식수 + 신규매수주식수)
                    c_stocks = buy_qty * int(buy_price)
                    n_stocks = ava_count * int(c_price_6)

                    buy_price = str( (math.floor((c_stocks + n_stocks) / (buy_qty + ava_count) * susu_rate) + up_price ))
                    print('평균 매수가격:' , buy_price)
                    
                    buy_qty = buy_qty + ava_count
                    
                    buy_price_rate = c_rate_6
                    sell_price = str( int(buy_price) + 50 )
                    
                                        
                    print('매수가 완료되어 5분 후 다시 시작합니다')
                    time.sleep(300)

            # 매도하기
            elif float(c_rate_6) < float(c_rate_5) and c_rate_pok_6 <= tot_sell_rate and masu == 1 and sell_price < c_price_6 :
            
                madobtn = driver.find_element_by_xpath('//*[@id="_0_F3000interact-with-order"]/ul[1]/li[2]/a') #매도가능 클릭
                driver.execute_script("arguments[0].click();", madobtn)
                time.sleep(1)
                
                #매도 가능 수량 찾기
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')            
                try :
                    madojan = int( soup.find('tbody','CI-GRID-BODY-TABLE-TBODY').find('td',{'name': 'od_unit'}).get_text().replace(',','') )
                except :
                    madojan = 0
                
                if madojan >= 1 :
                    print("매도합니다~~~")
                    time.sleep(0.5)
                    madotab = driver.find_element_by_xpath('//*[@id="W_0_F3000"]/div[2]/div/section[4]/div/ul[1]/li[2]/a') #매도 탭클릭
                    driver.execute_script("arguments[0].click();", madotab)
                    time.sleep(0.5)
                    pwd = driver.find_element_by_name('passwd')
                    pwd.clear()
                    time.sleep(0.5)
                    pwd.send_keys('이곳에 여러분의 HTS 암호를 입력하세요')
                    time.sleep(0.5)
                    driver.find_element_by_xpath('//*[@id="_0_F3000medo"]/fieldset/div[2]/input[2]').click()  # 조회 클릭
                    time.sleep(0.5)
                    radio2 = driver.find_element_by_id("_0_F3000medo_meme_gubun_radio2")   # 시장가 클릭
                    radio2.click()

                    time.sleep(0.5)
                    radio3 = driver.find_element_by_id('_0_F3000medo_meme_gubun_radio1')  # 보통 클릭
                    radio3.click( )
                    #driver.find_element_by_xpath('//*[@id="_0_F3000medo"]/fieldset/div[5]/div/input[3]').click() # 현재가보다 한단계 + 키 누르기
                    time.sleep(1)

                    #매도 수량 입력
                    order_count = driver.find_element_by_name('order_cnt')
                    order_count.clear( )
                    time.sleep(0.5)
                    order_count.send_keys(madojan)
                    #order_count.send_keys(1)
                    time.sleep(0.5)

                    madobtn2 = driver.find_element_by_xpath('//*[@id="_0_F3000medo"]/fieldset/div[9]/input')
                    driver.execute_script("arguments[0].click();", madobtn2) #매도 버튼 클릭
                    time.sleep(0.5)

                    madobtn3 = driver.find_element_by_xpath('//*[@id="orderConfirm__0_F3000medo"]/div[3]/div/button[1]/span') #팝업창 확인 클릭
                    driver.execute_script("arguments[0].click();", madobtn3)

                    masu = 0
                    time.sleep(1)

                    close1 = pyautogui.screenshot()
                    close2 = cv2.cvtColor(np.array(close1), cv2.COLOR_RGB2BGR)
                    cv2.imwrite("c:\\data\\image\\close_full_screen_1.jpg", close2)
                    time.sleep(1)

                    close_3 = cv2.imread('c:\\data\\image\\close.jpg', cv2.IMREAD_COLOR)  #확인버튼 

                    match_close = match_center_loc(close2, close_3)
                    pyautogui.moveTo(match_close)
                    pyautogui.click()
                    time.sleep(1)

                    print('%s 원으로 %s주 매도 신청 완료했습니다' %(c_price_6,madojan) )  
                    print('5분 뒤 다시 시작됩니다~잠시만 쉬세요~')
                    time.sleep(300)   # 매도 후 5 분 뒤 다시 매수 코드 작동
                    
                    buy_qty = 0
                    buy_price = '0'
                    buy_price_rate = 0.0
                    
            #손절하기
            elif son_gijun_price < son_price :
                madobtn = driver.find_element_by_xpath('//*[@id="_0_F3000interact-with-order"]/ul[1]/li[2]/a') #매도가능 클릭
                driver.execute_script("arguments[0].click();", madobtn)
                time.sleep(1)
                
                #매도 가능 수량 찾기
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')   
                try :
                    madojan = int( soup.find('tbody','CI-GRID-BODY-TABLE-TBODY').find('td',{'name': 'od_unit'}).get_text().replace(',','') )
                except :
                    madojan = 0
                
                if madojan >= 1 :
                    print("손절 매도합니다ㅠㅠㅠ")
                    time.sleep(0.5)
                    madotab = driver.find_element_by_xpath('//*[@id="W_0_F3000"]/div[2]/div/section[4]/div/ul[1]/li[2]/a') #매도 탭클릭
                    driver.execute_script("arguments[0].click();", madotab)
                    time.sleep(0.5)
                    pwd = driver.find_element_by_name('passwd')
                    pwd.clear()
                    time.sleep(0.5)
                    pwd.send_keys('이곳에 여러분의 HTS 암호를 입력하세요')
                    time.sleep(0.5)
                    driver.find_element_by_xpath('//*[@id="_0_F3000medo"]/fieldset/div[2]/input[2]').click()  # 조회 클릭
                    time.sleep(0.5)
                    radio2 = driver.find_element_by_id("_0_F3000medo_meme_gubun_radio2")   # 시장가 클릭
                    radio2.click()

                    time.sleep(0.5)
                    radio3 = driver.find_element_by_id('_0_F3000medo_meme_gubun_radio1')  # 보통 클릭
                    radio3.click( )
                    #driver.find_element_by_xpath('//*[@id="_0_F3000medo"]/fieldset/div[5]/div/input[3]').click() # 현재가보다 한단계 + 키 누르기
                    time.sleep(1)

                    #매도 수량 입력
                    order_count = driver.find_element_by_name('order_cnt')
                    order_count.clear( )
                    time.sleep(0.5)
                    order_count.send_keys(madojan)
                    #order_count.send_keys(1)
                    time.sleep(0.5)

                    madobtn2 = driver.find_element_by_xpath('//*[@id="_0_F3000medo"]/fieldset/div[9]/input')
                    driver.execute_script("arguments[0].click();", madobtn2) #매도 버튼 클릭
                    time.sleep(0.5)

                    madobtn3 = driver.find_element_by_xpath('//*[@id="orderConfirm__0_F3000medo"]/div[3]/div/button[1]/span') #팝업창 확인 클릭
                    driver.execute_script("arguments[0].click();", madobtn3)

                    masu = 0
                    time.sleep(1)

                    close1 = pyautogui.screenshot()
                    close2 = cv2.cvtColor(np.array(close1), cv2.COLOR_RGB2BGR)
                    cv2.imwrite("c:\\data\\image\\close_full_screen_1.jpg", close2)
                    time.sleep(1)

                    close_3 = cv2.imread('c:\\data\\image\\close.jpg', cv2.IMREAD_COLOR)  #확인버튼 

                    match_close = match_center_loc(close2, close_3)
                    pyautogui.moveTo(match_close)
                    pyautogui.click()
                    time.sleep(1)

                    print('%s 원으로 %s주 손절 매도 신청 완료했습니다' %(c_price_6,madojan) )  
                    print('5분 뒤 다시 시작됩니다~잠시만 쉬세요~')
                    time.sleep(300)   # 매도 후 5 분 뒤 다시 매수 코드 작동
                    buy_qty = 0
                    buy_price = '0'
                    buy_price_rate = 0.0
                    

        elif count > 6 and count%3==1 :
            time.sleep(1)
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            
            j_name = soup.find('div','j-codeSearcher codeSearcher inline').find('span').get_text()
            print('1.종목이름:',j_name)

            price_rate_all = soup.find('div','panel').find('dl','dl-layout four').find_all('dd')
            
            c_price_4 = price_rate_all[0].get_text().replace(",","")   #현재가격
            print('2.현재가격_%s: %s' %(count,c_price_4) )
            
            updown_price_4 = price_rate_all[1].get_text().replace(",","")   #현재가격
            print('3.가격변동금액_%s: %s' %(count,updown_price_4))

            c_rate_4 = price_rate_all[2].get_text().replace("%","") #등락율
            print('4.등락율_%s: %s' %(count,c_rate_4) )

            c_rate_pok_4 = float(c_rate_4) - float(c_rate_5)
            
            #손절기준가격지정
            son_gijun_price = int(buy_price) *  0.05   #현재가의 -5% 면 손절
            son_price = int(buy_price) - int(c_price_4)

            print("5.%s회 기준 최근 3회 변동폭 차이:%s" %(count,round(c_rate_pok_4,2) ) )

            time.sleep(1)
            driver.find_element_by_xpath('//*[@id="_0_F3000interact-with-order"]/ul[1]/li[2]/a').click( )  # 화면 움직임 만들기 위해 매도가능 탭 클릭
            time.sleep(1)
            driver.find_element_by_xpath('//*[@id="_0_F3000interact-with-order"]/ul[1]/li[1]/a').click( )  # 화면 움직임 만들기 위해 매수가능 탭 클릭
            time.sleep(0.5)
            
            price_gap = 0
                
            if c_price_4 == buy_price or buy_price == '0' :
                price_gap = 0
            elif c_price_4 < buy_price and c_price_4 >= str(math.ceil(int(buy_price) * 0.99)) :  # 현재 가격이 구입 가격보다 1% 미만 하락했을때
                price_gap = 1
            elif c_price_4 < buy_price and c_price_4 >= str(math.ceil(int(buy_price) * 0.97)) :  # 현재 가격이 구입 가격보다 1% - 3% 사이로 하락했을 때
                price_gap = 2
            elif c_price_4 < buy_price and c_price_4 >= str(math.ceil(int(buy_price) * 0.95)) :  # 현재 가격이 구입가격보다 3% - 5% 사이로 하락했을 때
                price_gap = 3
            elif c_price_4 < buy_price and c_price_4 <= str(math.ceil(int(buy_price) * 0.90))   : # 현재 가격이 구입가격보다 10% 이하 하락했을 때
                price_gap = 4

            elif c_price_4 > buy_price and c_price_4 <= str(math.ceil(int(buy_price) * 1.01)) :  # 현재 가격이 구입 가격보다 1% 미만 상승했을때
                price_gap = 5
            elif c_price_4 > buy_price and c_price_4 <= str(math.ceil(int(buy_price) * 1.03)) :  # 현재 가격이 구입 가격보다 1% - 3% 사이로 상승했을 때
                price_gap = 6
            elif c_price_4 > buy_price and c_price_4 <= str(math.ceil(int(buy_price) * 1.05)) :  # 현재 가격이 구입가격보다 3% - 5% 사이로 상승했을 때
                price_gap = 7
            elif c_price_4 > buy_price and c_price_4 >= str(math.ceil(int(buy_price) * 1.10))  : # 현재 가격이 구입가격보다 10% 사이로 상승했을 때
                price_gap = 8

            # 매수 / 매도 주문하기
            # 하락하다가 상승 전환시 매수하기
            #if c_rate_4 > c_rate_6 and abs(tot_6) >= tot_buy_rate :
            if float(c_rate_5) >= float(c_rate_6) and float(c_rate_6) < float(c_rate_4) and c_rate_pok_4  >= tot_buy_rate :
                time.sleep(1)
                #driver.find_element_by_xpath('//*[@id="W_0_F3000"]/div[2]/div/section[4]/div/ul[1]/li[1]').click( ) #매수버튼 클릭
                mesu_tab= driver.find_element_by_link_text('매수')  #매수 탭 클릭
                driver.execute_script("arguments[0].click();", mesu_tab)
                time.sleep(0.3)
                pwd = driver.find_element_by_name('passwd')
                pwd.clear()
                pwd.send_keys('이곳에 여러분의 HTS 암호를 입력하세요')
                #driver.find_element_by_xpath('//*[@id="_0_F3000mesu"]/fieldset/div[2]/input[2]').click()  # 조회 클릭
                jo_btn= driver.find_element_by_xpath('//*[@id="_0_F3000mesu"]/fieldset/div[2]/input[2]')
                driver.execute_script("arguments[0].click();", jo_btn)
                time.sleep(0.5)
                radio2 = driver.find_element_by_id("_0_F3000mesu_meme_gubun_radio2")   # 시장가 클릭
                radio2.click()

                time.sleep(0.5)
                radio3 = driver.find_element_by_id('_0_F3000mesu_meme_gubun_radio1')  # 지정가(보통) 클릭
                radio3.click( )

                #time.sleep(0.5)
                #driver.find_element_by_xpath('//*[@id="_0_F3000mesu"]/fieldset/div[5]/div/input[3]').click( ) # 현재가보다 한단계 + 키 누르기
                #time.sleep(0.3)
                #driver.find_element_by_xpath('//*[@id="_0_F3000mesu"]/fieldset/div[5]/div/input[3]').click( ) # 현재가보다 한단계 + 키 누르기
                #driver.find_element_by_xpath('//*[@id="_0_F3000mesu"]/fieldset/div[5]/div/input[1]').click() # 현재가보다 한단계 - 키 누르기
                time.sleep(0.5)

                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                time.sleep(0.5)

                qty_1 = soup.find('table','deal_limit_price j-subInfo_tbl hide-on-mobile').select('tbody > tr > td')   #최대수량    
                try :
                    ava_qty = int(qty_1[0].get_text())
                except :
                    ava_qty = 0
                print('최대구입가능수량:' ,ava_qty)
                
                if ava_qty >= 1 and price_gap == 0 and ava_qty >= 5:
                    ava_count = math.floor(ava_qty * 0.3)   # 30% 구입   
                elif price_gap == 1 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.1)  # 남아있는 금액에서 10% 구입:
                elif price_gap == 2 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.2)
                elif price_gap == 3 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.3)
                elif price_gap == 4 or ava_qty <= 5 :
                    ava_count = ava_qty
                elif price_gap == 5 and ava_qty >= 5:
                    ava_count = math.floor(ava_qty * 0.1)  # 남아있는 금액에서 30% 구입
                elif price_gap == 6 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.2)
                elif price_gap == 7 or price_gap == 8 or ava_qty <= 5   :
                    ava_count = ava_qty

                if ava_count >= 1 :

                    order_count = driver.find_element_by_name('order_cnt')
                    order_count.clear( )
                    time.sleep(0.5)
                    order_count.send_keys(ava_count)
                    #driver.find_element_by_xpath('//*[@id="_0_F3000mesu"]/fieldset/div[8]/input').click( )  #매수클릭
                    masu_btn= driver.find_element_by_xpath('//*[@id="_0_F3000mesu"]/fieldset/div[8]/input')
                    driver.execute_script("arguments[0].click();", masu_btn)
                    time.sleep(0.5)
                    #driver.find_element_by_xpath('//*[@id="orderConfirm__0_F3000mesu"]/div[3]/div/button[1]/span').click() #팝업창 매수 클릭
                    popup_masu_btn= driver.find_element_by_xpath('//*[@id="orderConfirm__0_F3000mesu"]/div[3]/div/button[1]/span')
                    driver.execute_script("arguments[0].click();", popup_masu_btn)
                    time.sleep(2)
                    driver.find_element_by_css_selector('span.ui-button-text').click()

                    close1 = pyautogui.screenshot()
                    close2 = cv2.cvtColor(np.array(close1), cv2.COLOR_RGB2BGR)
                    cv2.imwrite("c:\\data\\image\\close_full_screen_1.jpg", close2)
                    time.sleep(1)

                    close_3 = cv2.imread('c:\\data\\image\\close.jpg', cv2.IMREAD_COLOR)  #확인버튼 

                    match_close = match_center_loc(close2, close_3)
                    pyautogui.moveTo(match_close)
                    pyautogui.click()

                    print('%s 원으로 %s 수 매수 신청 완료했습니다' %(c_price_4,ava_count))
                    masu = 1
                    
                    # 평균 매수가 구하기
                    #공식 : (보유주식수 * 보유주식매수가격) + (신규매수수량 * 신규매수가격) / (보유주식수 + 신규매수주식수)
                    c_stocks = buy_qty * int(buy_price)
                    n_stocks = ava_count * int(c_price_4)

                    buy_price = str( (math.floor((c_stocks + n_stocks) / (buy_qty + ava_count) * susu_rate) + up_price ))
                    print('평균 매수가격:' , buy_price)
                    
                    buy_qty = buy_qty + ava_count                    
                    buy_price_rate = c_rate_4
                    sell_price = str( int(buy_price) + 50 )
                                                            
                    print('매수가 완료되어 5분 후 다시 시작합니다')
                    time.sleep(300)

            # 상승 상태 매수        
            elif float(c_rate_5) <= float(c_rate_6) and float(c_rate_6) < float(c_rate_4) and c_rate_pok_4 >= tot_buy_rate :
                time.sleep(1)
                #driver.find_element_by_xpath('//*[@id="W_0_F3000"]/div[2]/div/section[4]/div/ul[1]/li[1]').click( ) #매수버튼 클릭
                mesu_tab= driver.find_element_by_link_text('매수')  #매수 탭 클릭
                driver.execute_script("arguments[0].click();", mesu_tab)
                time.sleep(0.3)

                pwd = driver.find_element_by_name('passwd')
                pwd.clear()
                pwd.send_keys('이곳에 여러분의 HTS 암호를 입력하세요')
                #driver.find_element_by_xpath('//*[@id="_0_F3000mesu"]/fieldset/div[2]/input[2]').click()  # 조회 클릭
                jo_btn= driver.find_element_by_xpath('//*[@id="_0_F3000mesu"]/fieldset/div[2]/input[2]')
                driver.execute_script("arguments[0].click();", jo_btn)    # 에러의 원인이 되는 부분
                time.sleep(0.5)
                radio2 = driver.find_element_by_id("_0_F3000mesu_meme_gubun_radio2")   # 시장가 클릭
                radio2.click()

                time.sleep(0.5)
                radio3 = driver.find_element_by_id('_0_F3000mesu_meme_gubun_radio1')  # 지정가(보통) 클릭
                radio3.click( )

                time.sleep(0.5)

                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                time.sleep(0.5)

                qty_1 = soup.find('table','deal_limit_price j-subInfo_tbl hide-on-mobile').select('tbody > tr > td')   #최대수량    
                try :
                    ava_qty = int(qty_1[0].get_text())
                except :
                    ava_qty = 0
                print('최대구입가능수량:' ,ava_qty)
                    
                if ava_qty >= 1 and price_gap == 0 and ava_qty >= 5:
                    ava_count = math.floor(ava_qty * 0.3)   # 30% 구입   
                elif price_gap == 1 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.1)  # 남아있는 금액에서 10% 구입:
                elif price_gap == 2 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.2)
                elif price_gap == 3 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.3)
                elif price_gap == 4 or ava_qty <= 5 :
                    ava_count = ava_qty
                elif price_gap == 5 and ava_qty >= 5:
                    ava_count = math.floor(ava_qty * 0.1)  # 남아있는 금액에서 30% 구입
                elif price_gap == 6 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.2)
                elif price_gap == 7 or price_gap == 8 or ava_qty <= 5   :
                    ava_count = ava_qty

                if ava_count >= 1 :

                    order_count = driver.find_element_by_name('order_cnt')
                    order_count.clear( )
                    time.sleep(0.5)
                    order_count.send_keys(ava_count)
                    #driver.find_element_by_xpath('//*[@id="_0_F3000mesu"]/fieldset/div[8]/input').click( )  #매수클릭
                    masu_btn= driver.find_element_by_xpath('//*[@id="_0_F3000mesu"]/fieldset/div[8]/input')
                    driver.execute_script("arguments[0].click();", masu_btn)
                    time.sleep(0.5)
                    #driver.find_element_by_xpath('//*[@id="orderConfirm__0_F3000mesu"]/div[3]/div/button[1]/span').click() #팝업창 매수 클릭
                    popup_masu_btn= driver.find_element_by_xpath('//*[@id="orderConfirm__0_F3000mesu"]/div[3]/div/button[1]/span')
                    driver.execute_script("arguments[0].click();", popup_masu_btn)
                    time.sleep(2)
                    driver.find_element_by_css_selector('span.ui-button-text').click()

                    close1 = pyautogui.screenshot()
                    close2 = cv2.cvtColor(np.array(close1), cv2.COLOR_RGB2BGR)
                    cv2.imwrite("c:\\data\\image\\close_full_screen_1.jpg", close2)
                    time.sleep(1)

                    close_3 = cv2.imread('c:\\data\\image\\close.jpg', cv2.IMREAD_COLOR)  #확인버튼 

                    match_close = match_center_loc(close2, close_3)
                    pyautogui.moveTo(match_close)
                    pyautogui.click()

                    print('%s 원으로 %s 수 매수 신청 완료했습니다' %(c_price_4,ava_count))
                    masu = 1
                    
                    # 평균 매수가 구하기
                    #공식 : (보유주식수 * 보유주식매수가격) + (신규매수수량 * 신규매수가격) / (보유주식수 + 신규매수주식수)
                    c_stocks = buy_qty * int(buy_price)
                    n_stocks = ava_count * int(c_price_4)

                    buy_price = str( (math.floor((c_stocks + n_stocks) / (buy_qty + ava_count) * susu_rate) + up_price ))
                    print('평균 매수가격:' , buy_price)
                    
                    buy_qty = buy_qty + ava_count
                    
                    buy_price_rate = c_rate_4
                    sell_price = str( int(buy_price) + 50 )
                    
                    print('매수가 완료되어 5분 후 다시 시작합니다')
                    time.sleep(300)

            # 매도하기
            elif float(c_rate_4) < float(c_rate_6) and c_rate_pok_4 <= tot_sell_rate and masu == 1 and sell_price < c_price_4 :
            
                madobtn = driver.find_element_by_xpath('//*[@id="_0_F3000interact-with-order"]/ul[1]/li[2]/a') #매도가능 클릭
                driver.execute_script("arguments[0].click();", madobtn)
                time.sleep(1)
                
                #매도 가능 수량 찾기
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')            
                try :
                    madojan = int( soup.find('tbody','CI-GRID-BODY-TABLE-TBODY').find('td',{'name': 'od_unit'}).get_text().replace(',','') )
                except :
                    madojan = 0
                
                if madojan >= 1 :
                    print("매도합니다~~~")
                    time.sleep(0.5)
                    madotab = driver.find_element_by_xpath('//*[@id="W_0_F3000"]/div[2]/div/section[4]/div/ul[1]/li[2]/a') #매도 탭클릭
                    driver.execute_script("arguments[0].click();", madotab)
                    time.sleep(0.5)
                    pwd = driver.find_element_by_name('passwd')
                    pwd.clear()
                    time.sleep(0.5)
                    pwd.send_keys('이곳에 여러분의 HTS 암호를 입력하세요')
                    time.sleep(0.5)
                    driver.find_element_by_xpath('//*[@id="_0_F3000medo"]/fieldset/div[2]/input[2]').click()  # 조회 클릭
                    time.sleep(0.5)
                    radio2 = driver.find_element_by_id("_0_F3000medo_meme_gubun_radio2")   # 시장가 클릭
                    radio2.click()

                    time.sleep(0.5)
                    radio3 = driver.find_element_by_id('_0_F3000medo_meme_gubun_radio1')  # 보통 클릭
                    radio3.click( )
                    time.sleep(1)

                    #매도 수량 입력
                    order_count = driver.find_element_by_name('order_cnt')
                    order_count.clear( )
                    time.sleep(0.5)
                    order_count.send_keys(madojan)
                    #order_count.send_keys(1)
                    time.sleep(0.5)

                    madobtn2 = driver.find_element_by_xpath('//*[@id="_0_F3000medo"]/fieldset/div[9]/input')
                    driver.execute_script("arguments[0].click();", madobtn2) #매도 버튼 클릭
                    time.sleep(0.5)

                    madobtn3 = driver.find_element_by_xpath('//*[@id="orderConfirm__0_F3000medo"]/div[3]/div/button[1]/span') #팝업창 확인 클릭
                    driver.execute_script("arguments[0].click();", madobtn3)

                    masu = 0
                    time.sleep(1)

                    close1 = pyautogui.screenshot()
                    close2 = cv2.cvtColor(np.array(close1), cv2.COLOR_RGB2BGR)
                    cv2.imwrite("c:\\data\\image\\close_full_screen_1.jpg", close2)
                    time.sleep(1)

                    close_3 = cv2.imread('c:\\data\\image\\close.jpg', cv2.IMREAD_COLOR)  #확인버튼 

                    match_close = match_center_loc(close2, close_3)
                    pyautogui.moveTo(match_close)
                    pyautogui.click()
                    time.sleep(1)

                    print('%s 원으로 %s주 매도 신청 완료했습니다' %(c_price_4,madojan) )                                
                    print('5분 뒤 다시 시작됩니다~잠시만 쉬세요~')
                    time.sleep(300)   # 매도 후 5 분 뒤 다시 매수 코드 작동
                    
                    buy_qty = 0
                    buy_price = '0'
                    buy_price_rate = 0.0
                    
            #손절하기
            elif son_gijun_price < son_price :
                madobtn = driver.find_element_by_xpath('//*[@id="_0_F3000interact-with-order"]/ul[1]/li[2]/a') #매도가능 클릭
                driver.execute_script("arguments[0].click();", madobtn)
                time.sleep(1)
                
                #매도 가능 수량 찾기
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')   
                try :
                    madojan = int( soup.find('tbody','CI-GRID-BODY-TABLE-TBODY').find('td',{'name': 'od_unit'}).get_text().replace(',','') )
                except :
                    madojan = 0
                
                if madojan >= 1 :
                    print("손절 매도합니다ㅠㅠㅠ")
                    time.sleep(0.5)
                    madotab = driver.find_element_by_xpath('//*[@id="W_0_F3000"]/div[2]/div/section[4]/div/ul[1]/li[2]/a') #매도 탭클릭
                    driver.execute_script("arguments[0].click();", madotab)
                    time.sleep(0.5)
                    pwd = driver.find_element_by_name('passwd')
                    pwd.clear()
                    time.sleep(0.5)
                    pwd.send_keys('이곳에 여러분의 HTS 암호를 입력하세요')
                    time.sleep(0.5)
                    driver.find_element_by_xpath('//*[@id="_0_F3000medo"]/fieldset/div[2]/input[2]').click()  # 조회 클릭
                    time.sleep(0.5)
                    radio2 = driver.find_element_by_id("_0_F3000medo_meme_gubun_radio2")   # 시장가 클릭
                    radio2.click()

                    time.sleep(0.5)
                    radio3 = driver.find_element_by_id('_0_F3000medo_meme_gubun_radio1')  # 보통 클릭
                    radio3.click( )
                    time.sleep(1)

                    #매도 수량 입력
                    order_count = driver.find_element_by_name('order_cnt')
                    order_count.clear( )
                    time.sleep(0.5)
                    order_count.send_keys(madojan)
                    #order_count.send_keys(1)
                    time.sleep(0.5)

                    madobtn2 = driver.find_element_by_xpath('//*[@id="_0_F3000medo"]/fieldset/div[9]/input')
                    driver.execute_script("arguments[0].click();", madobtn2) #매도 버튼 클릭
                    time.sleep(0.5)

                    madobtn3 = driver.find_element_by_xpath('//*[@id="orderConfirm__0_F3000medo"]/div[3]/div/button[1]/span') #팝업창 확인 클릭
                    driver.execute_script("arguments[0].click();", madobtn3)

                    masu = 0
                    time.sleep(1)

                    close1 = pyautogui.screenshot()
                    close2 = cv2.cvtColor(np.array(close1), cv2.COLOR_RGB2BGR)
                    cv2.imwrite("c:\\data\\image\\close_full_screen_1.jpg", close2)
                    time.sleep(1)

                    close_3 = cv2.imread('c:\\data\\image\\close.jpg', cv2.IMREAD_COLOR)  #확인버튼 

                    match_close = match_center_loc(close2, close_3)
                    pyautogui.moveTo(match_close)
                    pyautogui.click()
                    time.sleep(1)
                    
                    buy_qty = 0
                    buy_price = '0'
                    buy_price_rate = 0.0
                    
                    print('%s 원으로 %s주 손절 매도 신청 완료했습니다' %(c_price_4,madojan) )
                    print('5분 뒤 다시 시작됩니다~잠시만 쉬세요~')
                    time.sleep(300)   # 매도 후 5 분 뒤 다시 매수 코드 작동
                    
        elif count > 6 and count%3==2 :
            time.sleep(1)
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')

            j_name = soup.find('div','j-codeSearcher codeSearcher inline').find('span').get_text()
            print('1.종목이름:',j_name)

            price_rate_all = soup.find('div','panel').find('dl','dl-layout four').find_all('dd')
            
            c_price_5 = price_rate_all[0].get_text().replace(",","")   #현재가격
            print('2.현재가격_%s: %s' %(count,c_price_5) )
            
            updown_price_5 = price_rate_all[1].get_text().replace(",","")   #현재가격
            print('3.가격변동금액_%s: %s' %(count,updown_price_5))

            c_rate_5 = price_rate_all[2].get_text().replace("%","") #등락율
            print('4.등락율_%s: %s' %(count,c_rate_5) )

            c_rate_pok_5 = float(c_rate_5) - float(c_rate_6)
            
            #손절기준가격지정
            son_gijun_price = int(buy_price) *  0.05   #현재가의 -5% 면 손절
            son_price = int(buy_price) - int(c_price_5)


            print("5.%s회 기준 최근 3회 변동폭 차이:%s" %(count,round(c_rate_pok_5 ,2) ) )

            time.sleep(0.5)
            driver.find_element_by_xpath('//*[@id="_0_F3000interact-with-order"]/ul[1]/li[2]/a').click( )  # 화면 움직임 만들기 위해 매도가능 탭 클릭
            time.sleep(0.5)
            driver.find_element_by_xpath('//*[@id="_0_F3000interact-with-order"]/ul[1]/li[1]/a').click( )  # 화면 움직임 만들기 위해 매수가능 탭 클릭
            time.sleep(0.5)
            
            price_gap = 0
                
            if c_price_5 == buy_price or buy_price == '0' :
                price_gap = 0
            elif c_price_5< buy_price and c_price_5 >= str(math.ceil(int(buy_price) * 0.99)) :  # 현재 가격이 구입 가격보다 1% 미만 하락했을때
                price_gap = 1
            elif c_price_5 < buy_price and c_price_5 >= str(math.ceil(int(buy_price) * 0.97)) :  # 현재 가격이 구입 가격보다 1% - 3% 사이로 하락했을 때
                price_gap = 2
            elif c_price_5 < buy_price and c_price_5 >= str(math.ceil(int(buy_price) * 0.95)) :  # 현재 가격이 구입가격보다 3% - 5% 사이로 하락했을 때
                price_gap = 3
            elif c_price_5 < buy_price and c_price_5 <= str(math.ceil(int(buy_price) * 0.90))   : # 현재 가격이 구입가격보다 10% 이하 하락했을 때
                price_gap = 4

            elif c_price_5 > buy_price and c_price_5 <= str(math.ceil(int(buy_price) * 1.01)) :  # 현재 가격이 구입 가격보다 1% 미만 상승했을때
                price_gap = 5
            elif c_price_5 > buy_price and c_price_5 <= str(math.ceil(int(buy_price) * 1.03)) :  # 현재 가격이 구입 가격보다 1% - 3% 사이로 상승했을 때
                price_gap = 6
            elif c_price_5 > buy_price and c_price_5 <= str(math.ceil(int(buy_price) * 1.05)) :  # 현재 가격이 구입가격보다 3% - 5% 사이로 상승했을 때
                price_gap = 7
            elif c_price_5 > buy_price and c_price_5 >= str(math.ceil(int(buy_price) * 1.10))  : # 현재 가격이 구입가격보다 10% 사이로 상승했을 때
                price_gap = 8
                
            # 매수 / 매도 주문하기
            # 하락하다가 상승 반전될 때 매수하기
            #if c_rate_5 > c_rate_4 and abs(tot_6) >= tot_buy_rate :
            if float(c_rate_6) >= float(c_rate_4) and float(c_rate_4) < float(c_rate_5) and c_rate_pok_5 >= tot_buy_rate :
                time.sleep(1)

                mesu_tab= driver.find_element_by_link_text('매수')  #매수 탭 클릭
                driver.execute_script("arguments[0].click();", mesu_tab)
                time.sleep(0.3)
                    
                pwd = driver.find_element_by_name('passwd')
                pwd.clear()
                pwd.send_keys('이곳에 여러분의 HTS 암호를 입력하세요')
                jo_btn= driver.find_element_by_xpath('//*[@id="_0_F3000mesu"]/fieldset/div[2]/input[2]')
                driver.execute_script("arguments[0].click();", jo_btn)
                time.sleep(0.5)
                radio2 = driver.find_element_by_id("_0_F3000mesu_meme_gubun_radio2")   # 시장가 클릭
                radio2.click()

                time.sleep(0.5)
                radio3 = driver.find_element_by_id('_0_F3000mesu_meme_gubun_radio1')  # 지정가(보통) 클릭
                radio3.click( )
                time.sleep(0.5)

                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                time.sleep(0.5)

                qty_1 = soup.find('table','deal_limit_price j-subInfo_tbl hide-on-mobile').select('tbody > tr > td')   #최대수량    
                try :
                    ava_qty = int(qty_1[0].get_text())
                except :
                    ava_qty = 0
                print('최대구입가능수량:' ,ava_qty)
                
                if ava_qty >= 1 and price_gap == 0 and ava_qty >= 5:
                    ava_count = math.floor(ava_qty * 0.3)   # 30% 구입   
                elif price_gap == 1 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.1)  # 남아있는 금액에서 10% 구입:
                elif price_gap == 2 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.2)
                elif price_gap == 3 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.3)
                elif price_gap == 4 or ava_qty <= 5 :
                    ava_count = ava_qty
                elif price_gap == 5 and ava_qty >= 5:
                    ava_count = math.floor(ava_qty * 0.1)  # 남아있는 금액에서 30% 구입
                elif price_gap == 6 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.2)
                elif price_gap == 7 or price_gap == 8 or ava_qty <= 5   :
                    ava_count = ava_qty

                if ava_count >= 1 :

                    order_count = driver.find_element_by_name('order_cnt')
                    order_count.clear( )
                    time.sleep(0.5)
                    order_count.send_keys(ava_count)
                    #driver.find_element_by_xpath('//*[@id="_0_F3000mesu"]/fieldset/div[8]/input').click( )  #매수클릭
                    masu_btn= driver.find_element_by_xpath('//*[@id="_0_F3000mesu"]/fieldset/div[8]/input')
                    driver.execute_script("arguments[0].click();", masu_btn)
                    time.sleep(0.5)
                    #driver.find_element_by_xpath('//*[@id="orderConfirm__0_F3000mesu"]/div[3]/div/button[1]/span').click() #팝업창 매수 클릭
                    popup_masu_btn= driver.find_element_by_xpath('//*[@id="orderConfirm__0_F3000mesu"]/div[3]/div/button[1]/span')
                    driver.execute_script("arguments[0].click();", popup_masu_btn)
                    time.sleep(2)
                    driver.find_element_by_css_selector('span.ui-button-text').click()

                    close1 = pyautogui.screenshot()
                    close2 = cv2.cvtColor(np.array(close1), cv2.COLOR_RGB2BGR)
                    cv2.imwrite("c:\\data\\image\\close_full_screen_1.jpg", close2)
                    time.sleep(1)

                    close_3 = cv2.imread('c:\\data\\image\\close.jpg', cv2.IMREAD_COLOR)  #확인버튼 

                    match_close = match_center_loc(close2, close_3)
                    pyautogui.moveTo(match_close)
                    pyautogui.click()

                    print('%s 원으로 %s 수 매수 신청 완료했습니다' %(c_price_5,ava_count))
                    masu = 1
                    
                    # 평균 매수가 구하기
                    #공식 : (보유주식수 * 보유주식매수가격) + (신규매수수량 * 신규매수가격) / (보유주식수 + 신규매수주식수)
                    c_stocks = buy_qty * int(buy_price)
                    n_stocks = ava_count * int(c_price_5)

                    buy_price = str( (math.floor((c_stocks + n_stocks) / (buy_qty + ava_count) * susu_rate) + up_price ))
                    print('평균 매수가격:' , buy_price)
                    
                    buy_qty = buy_qty + ava_count
                    
                    buy_price_rate = c_rate_5
                    sell_price = str( int(buy_price) + 50 )
                    
                    print('매수가 완료되어 5분 후 다시 시작합니다')
                    time.sleep(300)

            # 상승 상태 매수        
            elif float(c_rate_6) <= float(c_rate_4) and float(c_rate_4) < float(c_rate_5) and c_rate_pok_5 >= tot_buy_rate :
                time.sleep(1)
                mesu_tab= driver.find_element_by_link_text('매수')  #매수 탭 클릭
                driver.execute_script("arguments[0].click();", mesu_tab)
                time.sleep(0.3)

                pwd = driver.find_element_by_name('passwd')
                pwd.clear()
                pwd.send_keys('이곳에 여러분의 HTS 암호를 입력하세요')
                jo_btn= driver.find_element_by_xpath('//*[@id="_0_F3000mesu"]/fieldset/div[2]/input[2]')
                driver.execute_script("arguments[0].click();", jo_btn)
                time.sleep(0.5)
                radio2 = driver.find_element_by_id("_0_F3000mesu_meme_gubun_radio2")   # 시장가 클릭
                radio2.click()

                time.sleep(0.5)
                radio3 = driver.find_element_by_id('_0_F3000mesu_meme_gubun_radio1')  # 지정가(보통) 클릭
                radio3.click( )
                time.sleep(0.5)

                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                time.sleep(0.5)

                qty_1 = soup.find('table','deal_limit_price j-subInfo_tbl hide-on-mobile').select('tbody > tr > td')   #최대수량    
                try :
                    ava_qty = int(qty_1[0].get_text())
                except :
                    ava_qty = 0
                print('최대구입가능수량:' ,ava_qty)
                    
                if ava_qty >= 1 and price_gap == 0 and ava_qty >= 5:
                    ava_count = math.floor(ava_qty * 0.3)   # 30% 구입   
                elif price_gap == 1 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.1)  # 남아있는 금액에서 10% 구입:
                elif price_gap == 2 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.2)
                elif price_gap == 3 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.3)
                elif price_gap == 4 or ava_qty <= 5 :
                    ava_count = ava_qty
                elif price_gap == 5 and ava_qty >= 5:
                    ava_count = math.floor(ava_qty * 0.1)  # 남아있는 금액에서 30% 구입
                elif price_gap == 6 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.2)
                elif price_gap == 7 or price_gap == 8 or ava_qty <= 5   :
                    ava_count = ava_qty

                if ava_count >= 1 :
                    order_count = driver.find_element_by_name('order_cnt')
                    order_count.clear( )
                    time.sleep(0.5)
                    order_count.send_keys(ava_count)
                    masu_btn= driver.find_element_by_xpath('//*[@id="_0_F3000mesu"]/fieldset/div[8]/input')
                    driver.execute_script("arguments[0].click();", masu_btn)
                    time.sleep(0.5)

                    popup_masu_btn= driver.find_element_by_xpath('//*[@id="orderConfirm__0_F3000mesu"]/div[3]/div/button[1]/span')
                    driver.execute_script("arguments[0].click();", popup_masu_btn)
                    time.sleep(2)
                    driver.find_element_by_css_selector('span.ui-button-text').click()

                    close1 = pyautogui.screenshot()
                    close2 = cv2.cvtColor(np.array(close1), cv2.COLOR_RGB2BGR)
                    cv2.imwrite("c:\\data\\image\\close_full_screen_1.jpg", close2)
                    time.sleep(1)

                    close_3 = cv2.imread('c:\\data\\image\\close.jpg', cv2.IMREAD_COLOR)  #확인버튼 

                    match_close = match_center_loc(close2, close_3)
                    pyautogui.moveTo(match_close)
                    pyautogui.click()

                    print('%s 원으로 %s 수 매수 신청 완료했습니다' %(c_price_5,ava_count))
                    masu = 1
                    
                    # 평균 매수가 구하기
                    #공식 : (보유주식수 * 보유주식매수가격) + (신규매수수량 * 신규매수가격) / (보유주식수 + 신규매수주식수)
                    c_stocks = buy_qty * int(buy_price)
                    n_stocks = ava_count * int(c_price_5)

                    buy_price = str( (math.floor((c_stocks + n_stocks) / (buy_qty + ava_count) * susu_rate) + up_price ))
                    print('평균 매수가격:' , buy_price)
                    
                    buy_qty = buy_qty + ava_count
                    
                    buy_price_rate = c_rate_5
                    sell_price = str( int(buy_price) + 50 )
                    
                    print('매수가 완료되어 5분 후 다시 시작합니다')
                    time.sleep(300)

            # 매도하기
            elif float(c_rate_5) < float(c_rate_4) and c_rate_pok_5 <= tot_sell_rate and masu == 1 and sell_price < c_price_5 :
                madobtn = driver.find_element_by_xpath('//*[@id="_0_F3000interact-with-order"]/ul[1]/li[2]/a') #매도가능 클릭
                driver.execute_script("arguments[0].click();", madobtn)
                time.sleep(1)
                
                #매도 가능 수량 찾기
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')            
                try :
                    madojan = int( soup.find('tbody','CI-GRID-BODY-TABLE-TBODY').find('td',{'name': 'od_unit'}).get_text().replace(',','') )
                except :
                    madojan = 0
                
                if madojan >= 1 :
                    print("매도합니다~~~")
                    time.sleep(0.5)
                    madotab = driver.find_element_by_xpath('//*[@id="W_0_F3000"]/div[2]/div/section[4]/div/ul[1]/li[2]/a') #매도 탭클릭
                    driver.execute_script("arguments[0].click();", madotab)
                    time.sleep(0.5)
                    pwd = driver.find_element_by_name('passwd')
                    pwd.clear()
                    time.sleep(0.5)
                    pwd.send_keys('이곳에 여러분의 HTS 암호를 입력하세요')
                    time.sleep(0.5)
                    driver.find_element_by_xpath('//*[@id="_0_F3000medo"]/fieldset/div[2]/input[2]').click()  # 조회 클릭
                    time.sleep(0.5)
                    radio2 = driver.find_element_by_id("_0_F3000medo_meme_gubun_radio2")   # 시장가 클릭
                    radio2.click()

                    time.sleep(0.5)
                    radio3 = driver.find_element_by_id('_0_F3000medo_meme_gubun_radio1')  # 보통 클릭
                    radio3.click( )
                    #driver.find_element_by_xpath('//*[@id="_0_F3000medo"]/fieldset/div[5]/div/input[3]').click() # 현재가보다 한단계 + 키 누르기
                    time.sleep(1)

                    #매도 수량 입력
                    order_count = driver.find_element_by_name('order_cnt')
                    order_count.clear( )
                    time.sleep(0.5)
                    order_count.send_keys(madojan)
                    #order_count.send_keys(1)
                    time.sleep(0.5)

                    madobtn2 = driver.find_element_by_xpath('//*[@id="_0_F3000medo"]/fieldset/div[9]/input')
                    driver.execute_script("arguments[0].click();", madobtn2) #매도 버튼 클릭
                    time.sleep(0.5)

                    madobtn3 = driver.find_element_by_xpath('//*[@id="orderConfirm__0_F3000medo"]/div[3]/div/button[1]/span') #팝업창 확인 클릭
                    driver.execute_script("arguments[0].click();", madobtn3)

                    masu = 0
                    time.sleep(1)

                    close1 = pyautogui.screenshot()
                    close2 = cv2.cvtColor(np.array(close1), cv2.COLOR_RGB2BGR)
                    cv2.imwrite("c:\\data\\image\\close_full_screen_1.jpg", close2)
                    time.sleep(1)

                    close_3 = cv2.imread('c:\\data\\image\\close.jpg', cv2.IMREAD_COLOR)  #확인버튼 

                    match_close = match_center_loc(close2, close_3)
                    pyautogui.moveTo(match_close)
                    pyautogui.click()
                    time.sleep(1)

                    print('%s 원으로 %s주 매도 신청 완료했습니다' %(c_price_5,madojan) ) 
                    print('5분 뒤 다시 시작됩니다~잠시만 쉬세요~')
                    time.sleep(300)   # 매도 후 5 분 뒤 다시 매수 코드 작동
                    
                    buy_qty = 0
                    buy_price = '0'
                    buy_price_rate = 0.0
                    
            #손절하기
            elif son_gijun_price < son_price :
                madobtn = driver.find_element_by_xpath('//*[@id="_0_F3000interact-with-order"]/ul[1]/li[2]/a') #매도가능 클릭
                driver.execute_script("arguments[0].click();", madobtn)
                time.sleep(1)
                
                #매도 가능 수량 찾기
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')   
                try :
                    madojan = int( soup.find('tbody','CI-GRID-BODY-TABLE-TBODY').find('td',{'name': 'od_unit'}).get_text().replace(',','') )
                except :
                    madojan = 0
                
                if madojan >= 1 :
                    print("손절 매도합니다ㅠㅠㅠ")
                    time.sleep(0.5)
                    madotab = driver.find_element_by_xpath('//*[@id="W_0_F3000"]/div[2]/div/section[4]/div/ul[1]/li[2]/a') #매도 탭클릭
                    driver.execute_script("arguments[0].click();", madotab)
                    time.sleep(0.5)
                    pwd = driver.find_element_by_name('passwd')
                    pwd.clear()
                    time.sleep(0.5)
                    pwd.send_keys('이곳에 여러분의 HTS 암호를 입력하세요')
                    time.sleep(0.5)
                    driver.find_element_by_xpath('//*[@id="_0_F3000medo"]/fieldset/div[2]/input[2]').click()  # 조회 클릭
                    time.sleep(0.5)
                    radio2 = driver.find_element_by_id("_0_F3000medo_meme_gubun_radio2")   # 시장가 클릭
                    radio2.click()

                    time.sleep(0.5)
                    radio3 = driver.find_element_by_id('_0_F3000medo_meme_gubun_radio1')  # 보통 클릭
                    radio3.click( )
                    #driver.find_element_by_xpath('//*[@id="_0_F3000medo"]/fieldset/div[5]/div/input[3]').click() # 현재가보다 한단계 + 키 누르기
                    time.sleep(1)

                    #매도 수량 입력
                    order_count = driver.find_element_by_name('order_cnt')
                    order_count.clear( )
                    time.sleep(0.5)
                    order_count.send_keys(madojan)
                    #order_count.send_keys(1)
                    time.sleep(0.5)

                    madobtn2 = driver.find_element_by_xpath('//*[@id="_0_F3000medo"]/fieldset/div[9]/input')
                    driver.execute_script("arguments[0].click();", madobtn2) #매도 버튼 클릭
                    time.sleep(0.5)

                    madobtn3 = driver.find_element_by_xpath('//*[@id="orderConfirm__0_F3000medo"]/div[3]/div/button[1]/span') #팝업창 확인 클릭
                    driver.execute_script("arguments[0].click();", madobtn3)

                    masu = 0
                    time.sleep(1)

                    close1 = pyautogui.screenshot()
                    close2 = cv2.cvtColor(np.array(close1), cv2.COLOR_RGB2BGR)
                    cv2.imwrite("c:\\data\\image\\close_full_screen_1.jpg", close2)
                    time.sleep(1)

                    close_3 = cv2.imread('c:\\data\\image\\close.jpg', cv2.IMREAD_COLOR)  #확인버튼 

                    match_close = match_center_loc(close2, close_3)
                    pyautogui.moveTo(match_close)
                    pyautogui.click()
                    time.sleep(1)

                    print('%s 원으로 %s주 손절 매도 신청 완료했습니다' %(c_price_5,madojan) )   
                    print('5분 뒤 다시 시작됩니다~잠시만 쉬세요~')
                    time.sleep(300)   # 매도 후 5 분 뒤 다시 매수 코드 작동
                    
                    buy_qty = 0
                    buy_price = '0'
                    buy_price_rate = 0.0
                    

        elif count > 6 and count%3==0 :
            time.sleep(1)
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')

            j_name = soup.find('div','j-codeSearcher codeSearcher inline').find('span').get_text()
            print('1.종목이름:',j_name)

            price_rate_all = soup.find('div','panel').find('dl','dl-layout four').find_all('dd')
            
            c_price_6 = price_rate_all[0].get_text().replace(",","")   #현재가격
            print('2.현재가격_%s: %s' %(count,c_price_6) )
            
            updown_price_6 = price_rate_all[1].get_text().replace(",","")   #현재가격
            print('3.가격변동금액_%s: %s' %(count,updown_price_6))

            c_rate_6 = price_rate_all[2].get_text().replace("%","") #등락율
            print('4.등락율_%s: %s' %(count,c_rate_6) )

            c_rate_pok_6 = float(c_rate_6) - float(c_rate_4)
            
            #손절기준가격지정
            son_gijun_price = int(buy_price) *  0.05   #현재가의 -5% 면 손절
            son_price = int(buy_price) - int(c_price_6)

            print("5.%s회 기준 최근 3회 변동폭 차이:%s" %(count,round(c_rate_pok_6,2) ) )


            driver.find_element_by_xpath('//*[@id="_0_F3000interact-with-order"]/ul[1]/li[2]/a').click( )  # 화면 움직임 만들기 위해 매도가능 탭 클릭

            driver.find_element_by_xpath('//*[@id="_0_F3000interact-with-order"]/ul[1]/li[1]/a').click( )  # 화면 움직임 만들기 위해 매수가능 탭 클릭

            
            price_gap = 0
                
            if c_price_6 == buy_price or buy_price == '0' :
                price_gap = 0
            elif c_price_6 < buy_price and c_price_6 >= str(math.ceil(int(buy_price) * 0.99)) :  # 현재 가격이 구입 가격보다 1% 미만 하락했을때
                price_gap = 1
            elif c_price_6 < buy_price and c_price_6 >= str(math.ceil(int(buy_price) * 0.97)) :  # 현재 가격이 구입 가격보다 1% - 3% 사이로 하락했을 때
                price_gap = 2
            elif c_price_6 < buy_price and c_price_6 >= str(math.ceil(int(buy_price) * 0.95)) :  # 현재 가격이 구입가격보다 3% - 5% 사이로 하락했을 때
                price_gap = 3
            elif c_price_6 < buy_price and c_price_6 <= str(math.ceil(int(buy_price) * 0.90))   : # 현재 가격이 구입가격보다 10% 이하 하락했을 때
                price_gap = 4

            elif c_price_6 > buy_price and c_price_6 <= str(math.ceil(int(buy_price) * 1.01)) :  # 현재 가격이 구입 가격보다 1% 미만 상승했을때
                price_gap = 5
            elif c_price_6 > buy_price and c_price_6 <= str(math.ceil(int(buy_price) * 1.03)) :  # 현재 가격이 구입 가격보다 1% - 3% 사이로 상승했을 때
                price_gap = 6
            elif c_price_6 > buy_price and c_price_6 <= str(math.ceil(int(buy_price) * 1.05)) :  # 현재 가격이 구입가격보다 3% - 5% 사이로 상승했을 때
                price_gap = 7
            elif c_price_6 > buy_price and c_price_6 >= str(math.ceil(int(buy_price) * 1.10))  : # 현재 가격이 구입가격보다 10% 사이로 상승했을 때
                price_gap = 8
                
            # 매수 / 매도 주문하기
            # 하락하다가 상승 반전시 매수하기
            #if c_rate_6 > c_rate_5 and abs(tot_6) >= tot_buy_rate  :
            if float(c_rate_4) >= float(c_rate_5) and float(c_rate_5) < float(c_rate_6) and c_rate_pok_6  >= tot_buy_rate :
                time.sleep(1)
                #driver.find_element_by_xpath('//*[@id="W_0_F3000"]/div[2]/div/section[4]/div/ul[1]/li[1]').click( ) #매수버튼 클릭
                mesu_tab= driver.find_element_by_link_text('매수')  #매수 탭 클릭
                driver.execute_script("arguments[0].click();", mesu_tab)
                time.sleep(0.3)
                
                pwd = driver.find_element_by_name('passwd')
                pwd.clear()
                pwd.send_keys('이곳에 여러분의 HTS 암호를 입력하세요')
                #driver.find_element_by_xpath('//*[@id="_0_F3000mesu"]/fieldset/div[2]/input[2]').click()  # 조회 클릭
                jo_btn= driver.find_element_by_xpath('//*[@id="_0_F3000mesu"]/fieldset/div[2]/input[2]')
                driver.execute_script("arguments[0].click();", jo_btn)
                time.sleep(0.5)
                radio2 = driver.find_element_by_id("_0_F3000mesu_meme_gubun_radio2")   # 시장가 클릭
                radio2.click()

                time.sleep(0.5)
                radio3 = driver.find_element_by_id('_0_F3000mesu_meme_gubun_radio1')  # 지정가(보통) 클릭
                radio3.click( )
                time.sleep(0.5)

                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                time.sleep(0.5)

                qty_1 = soup.find('table','deal_limit_price j-subInfo_tbl hide-on-mobile').select('tbody > tr > td')   #최대수량    
                try :
                    ava_qty = int(qty_1[0].get_text())
                except :
                    ava_qty = 0
                print('최대구입가능수량:' ,ava_qty)
                
                if ava_qty >= 1 and price_gap == 0 and ava_qty >= 5:
                    ava_count = math.floor(ava_qty * 0.3)   # 30% 구입   
                elif price_gap == 1 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.1)  # 남아있는 금액에서 10% 구입:
                elif price_gap == 2 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.2)
                elif price_gap == 3 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.3)
                elif price_gap == 4 or ava_qty <= 5 :
                    ava_count = ava_qty
                elif price_gap == 5 and ava_qty >= 5:
                    ava_count = math.floor(ava_qty * 0.1)  # 남아있는 금액에서 30% 구입
                elif price_gap == 6 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.2)
                elif price_gap == 7 or price_gap == 8 or ava_qty <= 5   :
                    ava_count = ava_qty

                if ava_count >= 1 :

                    order_count = driver.find_element_by_name('order_cnt')
                    order_count.clear( )
                    time.sleep(0.5)
                    order_count.send_keys(ava_count)
                    #driver.find_element_by_xpath('//*[@id="_0_F3000mesu"]/fieldset/div[8]/input').click( )  #매수클릭
                    masu_btn= driver.find_element_by_xpath('//*[@id="_0_F3000mesu"]/fieldset/div[8]/input')
                    driver.execute_script("arguments[0].click();", masu_btn)
                    time.sleep(0.5)
                    #driver.find_element_by_xpath('//*[@id="orderConfirm__0_F3000mesu"]/div[3]/div/button[1]/span').click() #팝업창 매수 클릭
                    popup_masu_btn= driver.find_element_by_xpath('//*[@id="orderConfirm__0_F3000mesu"]/div[3]/div/button[1]/span')
                    driver.execute_script("arguments[0].click();", popup_masu_btn)
                    time.sleep(2)
                    driver.find_element_by_css_selector('span.ui-button-text').click()

                    close1 = pyautogui.screenshot()
                    close2 = cv2.cvtColor(np.array(close1), cv2.COLOR_RGB2BGR)
                    cv2.imwrite("c:\\data\\image\\close_full_screen_1.jpg", close2)
                    time.sleep(1)

                    close_3 = cv2.imread('c:\\data\\image\\close.jpg', cv2.IMREAD_COLOR)  #확인버튼 

                    match_close = match_center_loc(close2, close_3)
                    pyautogui.moveTo(match_close)
                    pyautogui.click()

                    print('%s 원으로 %s 수 매수 신청 완료했습니다' %(c_price_6,ava_count))
                    masu = 1
                    
                    # 평균 매수가 구하기
                    #공식 : (보유주식수 * 보유주식매수가격) + (신규매수수량 * 신규매수가격) / (보유주식수 + 신규매수주식수)
                    c_stocks = buy_qty * int(buy_price)
                    n_stocks = ava_count * int(c_price_6)

                    buy_price = str( (math.floor((c_stocks + n_stocks) / (buy_qty + ava_count) * susu_rate) + up_price ))
                    print('평균 매수가격:' , buy_price)
                    
                    buy_qty = buy_qty + ava_count
                    
                    buy_price_rate = c_rate_6
                    sell_price = str( int(buy_price) + 50 )
                    
                    print('매수가 완료되어 5분 후 다시 시작합니다')
                    time.sleep(300)

            # 상승 상태 매수        
            elif float(c_rate_4) <= float(c_rate_5) and float(c_rate_5) < float(c_rate_6) and c_rate_pok_6 >= tot_buy_rate  :
                time.sleep(1)
                #driver.find_element_by_xpath('//*[@id="W_0_F3000"]/div[2]/div/section[4]/div/ul[1]/li[1]').click( ) #매수버튼 클릭

                mesu_tab= driver.find_element_by_link_text('매수')  #매수 탭 클릭
                driver.execute_script("arguments[0].click();", mesu_tab)
                time.sleep(0.3)

                pwd = driver.find_element_by_name('passwd')
                pwd.clear()
                pwd.send_keys('이곳에 여러분의 HTS 암호를 입력하세요')
                #driver.find_element_by_xpath('//*[@id="_0_F3000mesu"]/fieldset/div[2]/input[2]').click()  # 조회 클릭
                jo_btn= driver.find_element_by_xpath('//*[@id="_0_F3000mesu"]/fieldset/div[2]/input[2]')
                driver.execute_script("arguments[0].click();", jo_btn)
                time.sleep(0.5)
                radio2 = driver.find_element_by_id("_0_F3000mesu_meme_gubun_radio2")   # 시장가 클릭
                radio2.click()

                time.sleep(0.5)
                radio3 = driver.find_element_by_id('_0_F3000mesu_meme_gubun_radio1')  # 지정가(보통) 클릭
                radio3.click( )
                time.sleep(0.5)

                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                time.sleep(0.5)

                qty_1 = soup.find('table','deal_limit_price j-subInfo_tbl hide-on-mobile').select('tbody > tr > td')   #최대수량    
                try :
                    ava_qty = int(qty_1[0].get_text())
                except :
                    ava_qty = 0
                print('최대구입가능수량:' ,ava_qty)
                    
                if ava_qty >= 1 and price_gap == 0 and ava_qty >= 5:
                    ava_count = math.floor(ava_qty * 0.3)   # 30% 구입   
                elif price_gap == 1 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.1)  # 남아있는 금액에서 10% 구입:
                elif price_gap == 2 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.2)
                elif price_gap == 3 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.3)
                elif price_gap == 4 or ava_qty <= 5 :
                    ava_count = ava_qty
                elif price_gap == 5 and ava_qty >= 5:
                    ava_count = math.floor(ava_qty * 0.1)  # 남아있는 금액에서 30% 구입
                elif price_gap == 6 and ava_qty >= 5 :
                    ava_count = math.floor(ava_qty * 0.2)
                elif price_gap == 7 or price_gap == 8 or ava_qty <= 5   :
                    ava_count = ava_qty

                if ava_count >= 1 :

                    order_count = driver.find_element_by_name('order_cnt')
                    order_count.clear( )
                    time.sleep(0.5)
                    order_count.send_keys(ava_count)
                    #driver.find_element_by_xpath('//*[@id="_0_F3000mesu"]/fieldset/div[8]/input').click( )  #매수클릭
                    masu_btn= driver.find_element_by_xpath('//*[@id="_0_F3000mesu"]/fieldset/div[8]/input')
                    driver.execute_script("arguments[0].click();", masu_btn)
                    time.sleep(0.5)
                    #driver.find_element_by_xpath('//*[@id="orderConfirm__0_F3000mesu"]/div[3]/div/button[1]/span').click() #팝업창 매수 클릭
                    popup_masu_btn= driver.find_element_by_xpath('//*[@id="orderConfirm__0_F3000mesu"]/div[3]/div/button[1]/span')
                    driver.execute_script("arguments[0].click();", popup_masu_btn)
                    time.sleep(2)
                    driver.find_element_by_css_selector('span.ui-button-text').click()

                    close1 = pyautogui.screenshot()
                    close2 = cv2.cvtColor(np.array(close1), cv2.COLOR_RGB2BGR)
                    cv2.imwrite("c:\\data\\image\\close_full_screen_1.jpg", close2)
                    time.sleep(1)

                    close_3 = cv2.imread('c:\\data\\image\\close.jpg', cv2.IMREAD_COLOR)  #확인버튼 

                    match_close = match_center_loc(close2, close_3)
                    pyautogui.moveTo(match_close)
                    pyautogui.click()

                    print('%s 원으로 %s 수 매수 신청 완료했습니다' %(c_price_6,ava_count))
                    masu = 1
                    
                    # 평균 매수가 구하기
                    #공식 : (보유주식수 * 보유주식매수가격) + (신규매수수량 * 신규매수가격) / (보유주식수 + 신규매수주식수)
                    c_stocks = buy_qty * int(buy_price)
                    n_stocks = ava_count * int(c_price_6)

                    buy_price = str( (math.floor((c_stocks + n_stocks) / (buy_qty + ava_count) * susu_rate) + up_price ))
                    print('평균 매수가격:' , buy_price)
                    
                    buy_qty = buy_qty + ava_count
                    
                    
                    buy_price_rate = c_rate_6
                    sell_price = str( int(buy_price) + 50 )
                    
                    print('매수가 완료되어 5분 후 다시 시작합니다')
                    time.sleep(300)

            # 매도하기
            elif float(c_rate_6) < float(c_rate_5) and c_rate_pok_6 <= tot_sell_rate and masu == 1 and sell_price < c_price_6 :
                madobtn = driver.find_element_by_xpath('//*[@id="_0_F3000interact-with-order"]/ul[1]/li[2]/a') #매도가능 클릭
                driver.execute_script("arguments[0].click();", madobtn)
                time.sleep(1)
                
                #매도 가능 수량 찾기
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')            
                try :
                    madojan = int( soup.find('tbody','CI-GRID-BODY-TABLE-TBODY').find('td',{'name': 'od_unit'}).get_text().replace(',','') )
                except :
                    madojan = 0
                
                if madojan >= 1 :
                    print("매도합니다~~~")
                    time.sleep(0.5)
                    madotab = driver.find_element_by_xpath('//*[@id="W_0_F3000"]/div[2]/div/section[4]/div/ul[1]/li[2]/a') #매도 탭클릭
                    driver.execute_script("arguments[0].click();", madotab)
                    time.sleep(0.5)
                    pwd = driver.find_element_by_name('passwd')
                    pwd.clear()
                    time.sleep(0.5)
                    pwd.send_keys('이곳에 여러분의 HTS 암호를 입력하세요')
                    time.sleep(0.5)
                    driver.find_element_by_xpath('//*[@id="_0_F3000medo"]/fieldset/div[2]/input[2]').click()  # 조회 클릭
                    time.sleep(0.5)
                    radio2 = driver.find_element_by_id("_0_F3000medo_meme_gubun_radio2")   # 시장가 클릭
                    radio2.click()

                    time.sleep(0.5)
                    radio3 = driver.find_element_by_id('_0_F3000medo_meme_gubun_radio1')  # 보통 클릭
                    radio3.click( )
                    #driver.find_element_by_xpath('//*[@id="_0_F3000medo"]/fieldset/div[5]/div/input[3]').click() # 현재가보다 한단계 + 키 누르기
                    time.sleep(1)

                    #매도 수량 입력
                    order_count = driver.find_element_by_name('order_cnt')
                    order_count.clear( )
                    time.sleep(0.5)
                    order_count.send_keys(madojan)
                    #order_count.send_keys(1)
                    time.sleep(0.5)

                    madobtn2 = driver.find_element_by_xpath('//*[@id="_0_F3000medo"]/fieldset/div[9]/input')
                    driver.execute_script("arguments[0].click();", madobtn2) #매도 버튼 클릭
                    time.sleep(0.5)

                    madobtn3 = driver.find_element_by_xpath('//*[@id="orderConfirm__0_F3000medo"]/div[3]/div/button[1]/span') #팝업창 확인 클릭
                    driver.execute_script("arguments[0].click();", madobtn3)

                    masu = 0
                    time.sleep(1)

                    close1 = pyautogui.screenshot()
                    close2 = cv2.cvtColor(np.array(close1), cv2.COLOR_RGB2BGR)
                    cv2.imwrite("c:\\data\\image\\close_full_screen_1.jpg", close2)
                    time.sleep(1)

                    close_3 = cv2.imread('c:\\data\\image\\close.jpg', cv2.IMREAD_COLOR)  #확인버튼 

                    match_close = match_center_loc(close2, close_3)
                    pyautogui.moveTo(match_close)
                    pyautogui.click()
                    time.sleep(1)

                    print('%s 원으로 %s주 매도 신청 완료했습니다' %(c_price_6,madojan) )  
                    print('5분 뒤 다시 시작됩니다~잠시만 쉬세요~')
                    time.sleep(300)   # 매도 후 5 분 뒤 다시 매수 코드 작동
                    
                    buy_qty = 0
                    buy_price = '0'
                    buy_price_rate = 0.0
                    
            #손절하기
            elif son_gijun_price < son_price :
                madobtn = driver.find_element_by_xpath('//*[@id="_0_F3000interact-with-order"]/ul[1]/li[2]/a') #매도가능 클릭
                driver.execute_script("arguments[0].click();", madobtn)
                time.sleep(1)
                
                #매도 가능 수량 찾기
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')   
                try :
                    madojan = int( soup.find('tbody','CI-GRID-BODY-TABLE-TBODY').find('td',{'name': 'od_unit'}).get_text().replace(',','') )
                except :
                    madojan = 0
                
                if madojan >= 1 :
                    print("손절 매도합니다ㅠㅠㅠ")
                    time.sleep(0.5)
                    madotab = driver.find_element_by_xpath('//*[@id="W_0_F3000"]/div[2]/div/section[4]/div/ul[1]/li[2]/a') #매도 탭클릭
                    driver.execute_script("arguments[0].click();", madotab)
                    time.sleep(0.5)
                    pwd = driver.find_element_by_name('passwd')
                    pwd.clear()
                    time.sleep(0.5)
                    pwd.send_keys('이곳에 여러분의 HTS 암호를 입력하세요')
                    time.sleep(0.5)
                    driver.find_element_by_xpath('//*[@id="_0_F3000medo"]/fieldset/div[2]/input[2]').click()  # 조회 클릭
                    time.sleep(0.5)
                    radio2 = driver.find_element_by_id("_0_F3000medo_meme_gubun_radio2")   # 시장가 클릭
                    radio2.click()

                    time.sleep(0.5)
                    radio3 = driver.find_element_by_id('_0_F3000medo_meme_gubun_radio1')  # 보통 클릭
                    radio3.click( )
                    #driver.find_element_by_xpath('//*[@id="_0_F3000medo"]/fieldset/div[5]/div/input[3]').click() # 현재가보다 한단계 + 키 누르기
                    time.sleep(1)

                    #매도 수량 입력
                    order_count = driver.find_element_by_name('order_cnt')
                    order_count.clear( )
                    time.sleep(0.5)
                    order_count.send_keys(madojan)
                    #order_count.send_keys(1)
                    time.sleep(0.5)

                    madobtn2 = driver.find_element_by_xpath('//*[@id="_0_F3000medo"]/fieldset/div[9]/input')
                    driver.execute_script("arguments[0].click();", madobtn2) #매도 버튼 클릭
                    time.sleep(0.5)

                    madobtn3 = driver.find_element_by_xpath('//*[@id="orderConfirm__0_F3000medo"]/div[3]/div/button[1]/span') #팝업창 확인 클릭
                    driver.execute_script("arguments[0].click();", madobtn3)

                    masu = 0
                    time.sleep(1)

                    close1 = pyautogui.screenshot()
                    close2 = cv2.cvtColor(np.array(close1), cv2.COLOR_RGB2BGR)
                    cv2.imwrite("c:\\data\\image\\close_full_screen_1.jpg", close2)
                    time.sleep(1)

                    close_3 = cv2.imread('c:\\data\\image\\close.jpg', cv2.IMREAD_COLOR)  #확인버튼 

                    match_close = match_center_loc(close2, close_3)
                    pyautogui.moveTo(match_close)
                    pyautogui.click()
                    time.sleep(1)

                    print('%s 원으로 %s주 손절 매도 신청 완료했습니다' %(c_price_6,madojan) ) 
                    print('5분 뒤 다시 시작됩니다~잠시만 쉬세요~')
                    time.sleep(300)   # 매도 후 5 분 뒤 다시 매수 코드 작동
                    
                    buy_qty = 0
                    buy_price = '0'
                    buy_price_rate = 0.0

        time.sleep(30)
        count += 1
        print("\n")